#HAM10000

##1주차

###데이터 로드

In [ ]:
# Kaggle API 설치
!pip install kaggle

# Kaggle API Key 설정
import json
import os
from google.colab import files
files.upload()  # kaggle.json 업로드

# Kaggle 설정
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 데이터셋 다운로드 (HAM10000 데이터셋)
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000

# 압축 해제
!unzip skin-cancer-mnist-ham10000.zip


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: ham10000_images_part_2/ISIC_0029325.jpg  
  inflating: ham10000_images_part_2/ISIC_0029326.jpg  
  inflating: ham10000_images_part_2/ISIC_0029327.jpg  
  inflating: ham10000_images_part_2/ISIC_0029328.jpg  
  inflating: ham10000_images_part_2/ISIC_0029329.jpg  
  inflating: ham10000_images_part_2/ISIC_0029330.jpg  
  inflating: ham10000_images_part_2/ISIC_0029331.jpg  
  inflating: ham10000_images_part_2/ISIC_0029332.jpg  
  inflating: ham10000_images_part_2/ISIC_0029333.jpg  
  inflating: ham10000_images_part_2/ISIC_0029334.jpg  
  inflating: ham10000_images_part_2/ISIC_0029335.jpg  
  inflating: ham10000_images_part_2/ISIC_0029336.jpg  
  inflating: ham10000_images_part_2/ISIC_0029337.jpg  
  inflating: ham10000_images_part_2/ISIC_0029338.jpg  
  inflating: ham10000_images_part_2/ISIC_0029339.jpg  
  inflating: ham10000_images_part_2/ISIC_0029340.jpg  
  inflating: ham10000_images_part_2/ISIC_0029341.jpg  
  inflating: ham10000_images_

In [ ]:
import os
import pandas as pd

# CSV 파일을 불러와서 DataFrame으로 저장
metadata_df = pd.read_csv('./HAM10000_metadata.csv')

# 이미지 폴더 경로
image_folder_1 = './HAM10000_images_part_1/'
image_folder_2 = './HAM10000_images_part_2/'

# 이미지 파일 경로 생성
metadata_df['image_path'] = metadata_df['image_id'].apply(
    lambda x: os.path.join(image_folder_1, f"{x}.jpg")
              if os.path.exists(os.path.join(image_folder_1, f"{x}.jpg"))
              else os.path.join(image_folder_2, f"{x}.jpg"))

# 메타데이터에 이미지 경로 추가 확인
print(metadata_df[['image_id', 'image_path']].head())


       image_id                                 image_path
0  ISIC_0027419  ./HAM10000_images_part_1/ISIC_0027419.jpg
1  ISIC_0025030  ./HAM10000_images_part_1/ISIC_0025030.jpg
2  ISIC_0026769  ./HAM10000_images_part_1/ISIC_0026769.jpg
3  ISIC_0025661  ./HAM10000_images_part_1/ISIC_0025661.jpg
4  ISIC_0031633  ./HAM10000_images_part_2/ISIC_0031633.jpg


###텐서플로우

In [ ]:
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# 고유한 클래스 추출 및 클래스 개수 계산
unique_classes = metadata_df['dx'].unique()
NUM_CLASSES = len(unique_classes)

print(f"Number of classes: {NUM_CLASSES}")
print(f"Classes: {unique_classes}")

# 메타데이터에서 데이터와 레이블 추출
train_df, val_df = train_test_split(metadata_df, test_size=0.2, stratify=metadata_df['dx'], random_state=42)

# ImageDataGenerator 설정
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

# 학습 데이터 생성기
train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,
    x_col='image_path',
    y_col='dx',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

# 검증 데이터 생성기
val_generator = datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=None,
    x_col='image_path',
    y_col='dx',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)


Number of classes: 7
Classes: ['bkl' 'nv' 'df' 'mel' 'vasc' 'bcc' 'akiec']
Found 6410 validated image filenames belonging to 7 classes.
Found 400 validated image filenames belonging to 7 classes.


In [ ]:
# EfficientNetB0 모델 사용
base_model = EfficientNetB0(include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

# CNN 모델 정의
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')  # CSV에서 불러온 클래스 개수 사용
])

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

###1차 수정 - 레이어 훈련 제약 수정

In [ ]:
# EfficientNetB0 모델 사용
base_model = EfficientNetB0(include_top=False, input_shape=(224, 224, 3))

# EfficientNetB0 모델의 마지막 몇 개 레이어만 훈련 가능하도록 설정
for layer in base_model.layers[:-20]:  # 마지막 20개 레이어는 학습 가능
    layer.trainable = False

# CNN 모델 정의
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.6496 - loss: 1.2504

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


201/201 ━━━━━━━━━━━━━━━━━━━━ 768s 4s/step - accuracy: 0.6497 - loss: 1.2500 - val_accuracy: 0.6325 - val_loss: 1.2119
Epoch 2/10
 39/201 ━━━━━━━━━━━━━━━━━━━━ 9:56 4s/step - accuracy: 0.6666 - loss: 1.1375

KeyboardInterrupt: 

###분리 검증

In [ ]:
# 원본 데이터에서 클래스별 분포 확인
original_class_distribution = metadata_df['dx'].value_counts()
print("Original class distribution:", original_class_distribution)


Original class distribution: dx
nv       6705
mel      1113
bkl      1099
bcc       514
akiec     327
vasc      142
df        115
Name: count, dtype: int64


In [ ]:
# 학습 및 검증 데이터 분리 후 클래스별 분포 확인
train_class_distribution = train_df['dx'].value_counts()
val_class_distribution = val_df['dx'].value_counts()

print("Train class distribution after split:", train_class_distribution)
print("Validation class distribution after split:", val_class_distribution)


Train class distribution after split: dx
nv       5364
mel       890
bkl       879
bcc       411
akiec     262
vasc      114
df         92
Name: count, dtype: int64
Validation class distribution after split: dx
nv       1341
mel       223
bkl       220
bcc       103
akiec      65
vasc       28
df         23
Name: count, dtype: int64


In [ ]:
import numpy as np

# 학습 및 검증 데이터의 클래스 분포 확인
train_class_counts = train_generator.classes
val_class_counts = val_generator.classes

# 클래스별 데이터 수 확인
train_class_distribution = np.bincount(train_class_counts)
val_class_distribution = np.bincount(val_class_counts)

# 학습과 검증 데이터의 클래스 분포 출력
print("Train class distribution:", train_class_distribution)
print("Validation class distribution:", val_class_distribution)

Train class distribution: [ 196  346  707   74  716 4274   97]
Validation class distribution: [ 13  25  45   3  53 253   8]


###파이토치

In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, ConcatDataset, Subset
from torchvision import transforms

# 사용자 정의 Dataset 클래스
class HAM10000Dataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['image_path']
        image = Image.open(img_path).convert("RGB")
        label = self.dataframe.iloc[idx]['label']

        if self.transform:
            image = self.transform(image)

        return image, label

# 병변 종류별로 라벨링 (7개의 라벨로 구분)
label_mapping = {
    'nv': 0,   # Melanocytic nevi
    'mel': 1,  # Melanoma
    'bkl': 2,  # Benign keratosis-like lesions
    'bcc': 3,  # Basal cell carcinoma
    'akiec': 4, # Actinic keratoses
    'vasc': 5, # Vascular lesions
    'df': 6    # Dermatofibroma
}

# 메타데이터 로드
metadata_df = pd.read_csv('./HAM10000_metadata.csv')
metadata_df['label'] = metadata_df['dx'].map(label_mapping)

# 학습 및 검증 데이터 분리 (80% 학습, 20% 검증)
train_df, val_df = train_test_split(metadata_df, test_size=0.2, stratify=metadata_df['dx'], random_state=42)

# 1. 데이터 증강 및 변환 정의
transform_original = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_augmented = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(20),
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 원본 데이터와 증강된 데이터셋 생성
original_dataset = HAM10000Dataset(train_df, transform=transform_original)
augmented_dataset = HAM10000Dataset(train_df, transform=transform_augmented)

# 원본 데이터와 증강된 데이터를 결합
combined_dataset = ConcatDataset([original_dataset, augmented_dataset])

# 2. 결합된 데이터셋을 학습용과 검증용으로 나누기
targets = [label for _, label in combined_dataset]
train_indices, val_indices = train_test_split(np.arange(len(combined_dataset)), test_size=0.2, stratify=targets)

train_dataset = Subset(combined_dataset, train_indices)
val_dataset = Subset(combined_dataset, val_indices)

# 3. 테스트 데이터셋 로드 (테스트는 증강하지 않음)
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 테스트 데이터셋 생성 (test_df는 별도로 분리된 테스트 데이터)
test_df = metadata_df.sample(frac=0.1, random_state=42)  # 예시로 10%를 테스트로 사용
test_dataset = HAM10000Dataset(test_df, transform=test_transform)

# 4. DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# 데이터 로더의 사용 예시
for images, labels in train_loader:
    print(f"Train batch - Images shape: {images.shape}, Labels shape: {labels.shape}")
    break

for images, labels in test_loader:
    print(f"Test batch - Images shape: {images.shape}, Labels shape: {labels.shape}")
    break


KeyError: 'image_path'

##2주차

###데이터 로드 + 데이터 증강 추가

In [ ]:
# Kaggle API 설치
!pip install kaggle

# Kaggle API Key 설정
import json
import os
from google.colab import files
files.upload()  # kaggle.json 업로드

# Kaggle 설정
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 데이터셋 다운로드 (HAM10000 데이터셋)
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000

# 압축 해제
!unzip skin-cancer-mnist-ham10000.zip


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: ham10000_images_part_2/ISIC_0029325.jpg  
  inflating: ham10000_images_part_2/ISIC_0029326.jpg  
  inflating: ham10000_images_part_2/ISIC_0029327.jpg  
  inflating: ham10000_images_part_2/ISIC_0029328.jpg  
  inflating: ham10000_images_part_2/ISIC_0029329.jpg  
  inflating: ham10000_images_part_2/ISIC_0029330.jpg  
  inflating: ham10000_images_part_2/ISIC_0029331.jpg  
  inflating: ham10000_images_part_2/ISIC_0029332.jpg  
  inflating: ham10000_images_part_2/ISIC_0029333.jpg  
  inflating: ham10000_images_part_2/ISIC_0029334.jpg  
  inflating: ham10000_images_part_2/ISIC_0029335.jpg  
  inflating: ham10000_images_part_2/ISIC_0029336.jpg  
  inflating: ham10000_images_part_2/ISIC_0029337.jpg  
  inflating: ham10000_images_part_2/ISIC_0029338.jpg  
  inflating: ham10000_images_part_2/ISIC_0029339.jpg  
  inflating: ham10000_images_part_2/ISIC_0029340.jpg  
  inflating: ham10000_images_part_2/ISIC_0029341.jpg  
  inflating: ham10000_images_

In [ ]:
import os
import pandas as pd

# CSV 파일을 불러와서 DataFrame으로 저장
metadata_df = pd.read_csv('./HAM10000_metadata.csv')

# 이미지 폴더 경로
image_folder_1 = './HAM10000_images_part_1/'
image_folder_2 = './HAM10000_images_part_2/'

# 이미지 파일 경로 생성
metadata_df['image_path'] = metadata_df['image_id'].apply(
    lambda x: os.path.join(image_folder_1, f"{x}.jpg")
              if os.path.exists(os.path.join(image_folder_1, f"{x}.jpg"))
              else os.path.join(image_folder_2, f"{x}.jpg"))

# 메타데이터에 이미지 경로 추가 확인
print(metadata_df[['image_id', 'image_path']].head())


       image_id                                 image_path
0  ISIC_0027419  ./HAM10000_images_part_1/ISIC_0027419.jpg
1  ISIC_0025030  ./HAM10000_images_part_1/ISIC_0025030.jpg
2  ISIC_0026769  ./HAM10000_images_part_1/ISIC_0026769.jpg
3  ISIC_0025661  ./HAM10000_images_part_1/ISIC_0025661.jpg
4  ISIC_0031633  ./HAM10000_images_part_2/ISIC_0031633.jpg


In [ ]:
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# 고유한 클래스 추출 및 클래스 개수 계산
unique_classes = metadata_df['dx'].unique()
NUM_CLASSES = len(unique_classes)

print(f"Number of classes: {NUM_CLASSES}")
print(f"Classes: {unique_classes}")

# 메타데이터에서 데이터와 레이블 추출 (train/test로 변경)
train_df, test_df = train_test_split(metadata_df, test_size=0.2, stratify=metadata_df['dx'], random_state=42)


# 학습 데이터용 ImageDataGenerator (데이터 증강 + 학습/검증 분리)
train_datagen = ImageDataGenerator(
    rescale=1./255,                # 정규화
    rotation_range=40,             # 이미지를 랜덤하게 40도까지 회전
    width_shift_range=0.2,         # 이미지의 가로 위치를 최대 20%까지 이동
    height_shift_range=0.2,        # 이미지의 세로 위치를 최대 20%까지 이동
    shear_range=0.2,               # 랜덤한 전단 변환 (shear)
    zoom_range=0.2,                # 랜덤하게 이미지 크기를 20%까지 확대/축소
    horizontal_flip=True,          # 이미지를 좌우로 랜덤하게 반전
    fill_mode='nearest',           # 이미지 이동 시 생기는 빈 공간을 채우는 방식
    validation_split=0.2           # 학습 데이터 중 20%를 검증용으로 분리
)

# 테스트 데이터용 ImageDataGenerator (데이터 증강 없음, 정규화만 적용)
test_datagen = ImageDataGenerator(
    rescale=1./255  # 테스트 데이터는 증강 없이 정규화만 적용
)

# 학습 데이터 생성기 (학습용)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,
    x_col='image_path',
    y_col='dx',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',  # 학습용 데이터
    shuffle=True
)

# 학습 데이터 생성기 (검증용)
val_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,
    x_col='image_path',
    y_col='dx',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',  # 검증용 데이터
    shuffle=True
)

# 테스트 데이터 생성기
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=None,
    x_col='image_path',
    y_col='dx',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # 테스트 데이터는 shuffle하지 않음
)

# 생성기 출력 테스트
print(f"Training data shape: {train_generator.image_shape}")
print(f"Validation data shape: {val_generator.image_shape}")
print(f"Test data shape: {test_generator.image_shape}")

Number of classes: 7
Classes: ['bkl' 'nv' 'df' 'mel' 'vasc' 'bcc' 'akiec']
Found 6410 validated image filenames belonging to 7 classes.
Found 1602 validated image filenames belonging to 7 classes.
Found 2003 validated image filenames belonging to 7 classes.
Training data shape: (224, 224, 3)
Validation data shape: (224, 224, 3)
Test data shape: (224, 224, 3)


###모델 학습

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# EfficientNetB0 모델 정의 (ImageNet 사전 학습된 가중치 사용)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모델을 쌓음
model = Sequential([
    base_model,  # EfficientNetB0 기반 모델
    GlobalAveragePooling2D(),  # 전역 평균 풀링
    Dense(128, activation='relu'),  # 중간 레이어
    Dense(NUM_CLASSES, activation='softmax')  # 출력 레이어: 클래스 수만큼 softmax
])

# 모델 컴파일 (accuracy와 함께 AUC 추가)
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])  # AUC 추가

# 모델 학습
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,  # 원하는 에포크 수로 조정 가능
    verbose=1
)


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


201/201 ━━━━━━━━━━━━━━━━━━━━ 331s 1s/step - accuracy: 0.7106 - auc: 0.9397 - loss: 0.8343 - val_accuracy: 0.0406 - val_auc: 0.4508 - val_loss: 2.3468
Epoch 2/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 156s 760ms/step - accuracy: 0.7820 - auc: 0.9703 - loss: 0.5881 - val_accuracy: 0.6792 - val_auc: 0.8176 - val_loss: 2.9091
Epoch 3/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 198s 743ms/step - accuracy: 0.8276 - auc: 0.9777 - loss: 0.5088 - val_accuracy: 0.6804 - val_auc: 0.8319 - val_loss: 1.3768
Epoch 4/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 156s 754ms/step - accuracy: 0.8342 - auc: 0.9819 - loss: 0.4520 - val_accuracy: 0.7097 - val_auc: 0.9406 - val_loss: 0.9555
Epoch 5/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 200s 750ms/step - accuracy: 0.8489 - auc: 0.9837 - loss: 0.4258 - val_accuracy: 0.7185 - val_auc: 0.9247 - val_loss: 1.2615
Epoch 6/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 156s 746ms/step - accuracy: 0.8551 - auc: 0.9846 - loss: 0.4079 - val_accuracy: 0.8034 - val_auc: 0.9644 - val_loss: 0.6723
Epoch 7/10
201/201 ━━━━━━━━━━━

In [ ]:
# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test AUC: {test_auc}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 30s 478ms/step - accuracy: 0.7928 - auc: 0.9532 - loss: 0.8157
Test accuracy: 0.7973040342330933
Test AUC: 0.9569269418716431


##3주차

##3주차 - 데이터 필터링

###데이터 로드 + 데이터 증강 + 필터링 추가

In [ ]:
# Kaggle API 설치
!pip install kaggle

# Kaggle API Key 설정
import json
import os
from google.colab import files
files.upload()  # kaggle.json 업로드

# Kaggle 설정
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 데이터셋 다운로드 (HAM10000 데이터셋)
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000

# 압축 해제
!unzip skin-cancer-mnist-ham10000.zip


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: ham10000_images_part_2/ISIC_0029325.jpg  
  inflating: ham10000_images_part_2/ISIC_0029326.jpg  
  inflating: ham10000_images_part_2/ISIC_0029327.jpg  
  inflating: ham10000_images_part_2/ISIC_0029328.jpg  
  inflating: ham10000_images_part_2/ISIC_0029329.jpg  
  inflating: ham10000_images_part_2/ISIC_0029330.jpg  
  inflating: ham10000_images_part_2/ISIC_0029331.jpg  
  inflating: ham10000_images_part_2/ISIC_0029332.jpg  
  inflating: ham10000_images_part_2/ISIC_0029333.jpg  
  inflating: ham10000_images_part_2/ISIC_0029334.jpg  
  inflating: ham10000_images_part_2/ISIC_0029335.jpg  
  inflating: ham10000_images_part_2/ISIC_0029336.jpg  
  inflating: ham10000_images_part_2/ISIC_0029337.jpg  
  inflating: ham10000_images_part_2/ISIC_0029338.jpg  
  inflating: ham10000_images_part_2/ISIC_0029339.jpg  
  inflating: ham10000_images_part_2/ISIC_0029340.jpg  
  inflating: ham10000_images_part_2/ISIC_0029341.jpg  
  inflating: ham10000_images_

In [ ]:
import os
import pandas as pd

# CSV 파일을 불러와서 DataFrame으로 저장
metadata_df = pd.read_csv('./HAM10000_metadata.csv')

# 이미지 폴더 경로
image_folder_1 = './HAM10000_images_part_1/'
image_folder_2 = './HAM10000_images_part_2/'

# 이미지 파일 경로 생성
metadata_df['image_path'] = metadata_df['image_id'].apply(
    lambda x: os.path.join(image_folder_1, f"{x}.jpg")
              if os.path.exists(os.path.join(image_folder_1, f"{x}.jpg"))
              else os.path.join(image_folder_2, f"{x}.jpg"))

# 메타데이터에 이미지 경로 추가 확인
print(metadata_df[['image_id', 'image_path']].head())


       image_id                                 image_path
0  ISIC_0027419  ./HAM10000_images_part_1/ISIC_0027419.jpg
1  ISIC_0025030  ./HAM10000_images_part_1/ISIC_0025030.jpg
2  ISIC_0026769  ./HAM10000_images_part_1/ISIC_0026769.jpg
3  ISIC_0025661  ./HAM10000_images_part_1/ISIC_0025661.jpg
4  ISIC_0031633  ./HAM10000_images_part_2/ISIC_0031633.jpg


In [ ]:
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# 고유한 클래스 추출 및 클래스 개수 계산
class_counts = metadata_df['dx'].value_counts()  # 클래스별 데이터 개수 계산
top_4_classes = class_counts.head(4).index  # 상위 4개 클래스 추출

print(f"Top 4 classes: {top_4_classes}")

# 상위 4개 클래스만 포함한 데이터프레임 필터링
filtered_df = metadata_df[metadata_df['dx'].isin(top_4_classes)]

# 메타데이터에서 데이터와 레이블 추출 (train/test로 변경)
train_df, test_df = train_test_split(filtered_df, test_size=0.2, stratify=filtered_df['dx'], random_state=42)

# 학습 데이터용 ImageDataGenerator (데이터 증강 + 학습/검증 분리)
train_datagen = ImageDataGenerator(
    rescale=1./255,                # 정규화
    rotation_range=40,             # 이미지를 랜덤하게 40도까지 회전
    width_shift_range=0.2,         # 이미지의 가로 위치를 최대 20%까지 이동
    height_shift_range=0.2,        # 이미지의 세로 위치를 최대 20%까지 이동
    shear_range=0.2,               # 랜덤한 전단 변환 (shear)
    zoom_range=0.2,                # 랜덤하게 이미지 크기를 20%까지 확대/축소
    horizontal_flip=True,          # 이미지를 좌우로 랜덤하게 반전
    fill_mode='nearest',           # 이미지 이동 시 생기는 빈 공간을 채우는 방식
    validation_split=0.2           # 학습 데이터 중 20%를 검증용으로 분리
)

# 테스트 데이터용 ImageDataGenerator (데이터 증강 없음, 정규화만 적용)
test_datagen = ImageDataGenerator(
    rescale=1./255  # 테스트 데이터는 증강 없이 정규화만 적용
)

# 학습 데이터 생성기 (학습용)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,
    x_col='image_path',
    y_col='dx',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',  # 학습용 데이터
    shuffle=True
)

# 학습 데이터 생성기 (검증용)
val_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,
    x_col='image_path',
    y_col='dx',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',  # 검증용 데이터
    shuffle=True
)

# 테스트 데이터 생성기
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=None,
    x_col='image_path',
    y_col='dx',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # 테스트 데이터는 shuffle하지 않음
)

# 생성기 출력 테스트
print(f"Training data shape: {train_generator.image_shape}")
print(f"Validation data shape: {val_generator.image_shape}")
print(f"Test data shape: {test_generator.image_shape}")

Top 4 classes: Index(['nv', 'mel', 'bkl', 'bcc'], dtype='object', name='dx')
Found 6036 validated image filenames belonging to 4 classes.
Found 1508 validated image filenames belonging to 4 classes.
Found 1887 validated image filenames belonging to 4 classes.
Training data shape: (224, 224, 3)
Validation data shape: (224, 224, 3)
Test data shape: (224, 224, 3)


###모델 학습 + 클래스 불균형 해소를 위한 가중치 추가

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# 클래스별 샘플 수에 따른 가중치 계산
class_counts = filtered_df['dx'].value_counts()  # 각 클래스의 샘플 수
total_samples = len(filtered_df)  # 전체 샘플 수
class_weights = {i: total_samples / class_counts[cls] for i, cls in enumerate(top_4_classes)}

print(f"Class weights: {class_weights}")

# EfficientNetB0 모델 정의 (ImageNet 사전 학습된 가중치 사용)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모델을 쌓음
model = Sequential([
    base_model,  # EfficientNetB0 기반 모델
    GlobalAveragePooling2D(),  # 전역 평균 풀링
    Dense(128, activation='relu'),  # 중간 레이어
    Dense(len(top_4_classes), activation='softmax')  # 출력 레이어: 4개 클래스
])
# 모델 컴파일 (accuracy와 함께 AUC 추가)
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])  # AUC 추가

# 모델 학습
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,  # 원하는 에포크 수로 조정 가능
    verbose=1,
    class_weight=class_weights  # 클래스 가중치 추가
)


Class weights: {0: 1.4065622669649516, 1: 8.473495058400719, 2: 8.581437670609645, 3: 18.348249027237355}
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


189/189 ━━━━━━━━━━━━━━━━━━━━ 369s 1s/step - accuracy: 0.7327 - auc: 0.9104 - loss: 6.6882 - val_accuracy: 0.7308 - val_auc: 0.8240 - val_loss: 1.6342
Epoch 2/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 209s 785ms/step - accuracy: 0.7887 - auc: 0.9489 - loss: 4.6407 - val_accuracy: 0.7308 - val_auc: 0.8395 - val_loss: 1.6477
Epoch 3/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 198s 775ms/step - accuracy: 0.8167 - auc: 0.9605 - loss: 3.9778 - val_accuracy: 0.6731 - val_auc: 0.8326 - val_loss: 1.3010
Epoch 4/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 208s 788ms/step - accuracy: 0.8308 - auc: 0.9647 - loss: 3.6918 - val_accuracy: 0.7500 - val_auc: 0.9227 - val_loss: 0.9254
Epoch 5/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 151s 780ms/step - accuracy: 0.8382 - auc: 0.9683 - loss: 3.5883 - val_accuracy: 0.8031 - val_auc: 0.9516 - val_loss: 0.6575
Epoch 6/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 153s 793ms/step - accuracy: 0.8550 - auc: 0.9745 - loss: 3.1855 - val_accuracy: 0.8090 - val_auc: 0.9558 - val_loss: 0.5722
Epoch 7/10
189/189 ━━━━━━━━━━━

In [ ]:
# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test AUC: {test_auc}")

59/59 ━━━━━━━━━━━━━━━━━━━━ 35s 598ms/step - accuracy: 0.7187 - auc: 0.9166 - loss: 0.8496
Test accuracy: 0.7069422602653503
Test AUC: 0.9124119281768799


###모델 학습. 클래스 불균형을 위한 가중치 추가 없음

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# EfficientNetB0 모델 정의 (ImageNet 사전 학습된 가중치 사용)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모델을 쌓음
model = Sequential([
    base_model,  # EfficientNetB0 기반 모델
    GlobalAveragePooling2D(),  # 전역 평균 풀링
    Dense(128, activation='relu'),  # 중간 레이어
    Dense(len(top_4_classes), activation='softmax')  # 출력 레이어: 4개 클래스
])
# 모델 컴파일 (accuracy와 함께 AUC 추가)
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])  # AUC 추가

# 모델 학습
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,  # 원하는 에포크 수로 조정 가능
    verbose=1,
)

Epoch 1/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 295s 1s/step - accuracy: 0.7368 - auc: 0.9150 - loss: 0.7022 - val_accuracy: 0.7308 - val_auc: 0.8413 - val_loss: 1.0064
Epoch 2/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 154s 796ms/step - accuracy: 0.8009 - auc: 0.9574 - loss: 0.4965 - val_accuracy: 0.3057 - val_auc: 0.5405 - val_loss: 1.3914
Epoch 3/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 152s 785ms/step - accuracy: 0.8403 - auc: 0.9683 - loss: 0.4250 - val_accuracy: 0.4794 - val_auc: 0.7506 - val_loss: 1.1842
Epoch 4/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 167s 855ms/step - accuracy: 0.8485 - auc: 0.9705 - loss: 0.4082 - val_accuracy: 0.7454 - val_auc: 0.8968 - val_loss: 1.2973
Epoch 5/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 153s 783ms/step - accuracy: 0.8603 - auc: 0.9757 - loss: 0.3673 - val_accuracy: 0.7924 - val_auc: 0.9475 - val_loss: 0.6341
Epoch 6/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 201s 785ms/step - accuracy: 0.8797 - auc: 0.9816 - loss: 0.3191 - val_accuracy: 0.7924 - val_auc: 0.9455 - val_loss: 0.7200
Epoch 7/10
189/189 

##3주차 - 무거운 다른 모델들 (가중치 없음) (성능 안좋아서 돌리다가 중단함)

###VGG16

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# VGG16 모델 정의 (ImageNet 사전 학습된 가중치 사용)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모델을 쌓음
model = Sequential([
    base_model,  # VGG16 기반 모델
    GlobalAveragePooling2D(),  # 전역 평균 풀링
    Dense(128, activation='relu'),  # 중간 레이어
    Dense(len(top_4_classes), activation='softmax')  # 출력 레이어: 4개 클래스
])

# 모델 컴파일 (accuracy와 함께 AUC 추가)
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])  # AUC 추가

# 모델 학습
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,  # 원하는 에포크 수로 조정 가능
    verbose=1,
)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Epoch 1/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 304s 1s/step - accuracy: 0.6754 - auc: 0.8037 - loss: 1.2074 - val_accuracy: 0.7308 - val_auc: 0.8745 - val_loss: 0.8699
Epoch 2/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 189s 963ms/step - accuracy: 0.7055 - auc: 0.8746 - loss: 0.8319 - val_accuracy: 0.7308 - val_auc: 0.8915 - val_loss: 0.7635
Epoch 3/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 209s 1s/step - accuracy: 0.7087 - auc: 0.8869 - loss: 0.7877 - val_accuracy: 0.7308 - val_auc: 0.9027 - val_loss: 0.7459
Epoch 4/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 787ms/step - accuracy: 0.6973 - auc: 0.8823 - loss: 0.7999

KeyboardInterrupt: 

In [ ]:
# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test AUC: {test_auc}")

###ResNet50

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# ResNet50 모델 정의 (ImageNet 사전 학습된 가중치 사용)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모델을 쌓음
model = Sequential([
    base_model,  # ResNet50 기반 모델
    GlobalAveragePooling2D(),  # 전역 평균 풀링
    Dense(128, activation='relu'),  # 중간 레이어
    Dense(len(top_4_classes), activation='softmax')  # 출력 레이어: 4개 클래스
])

# 모델 컴파일 (accuracy와 함께 AUC 추가)
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])  # AUC 추가

# 모델 학습
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,  # 원하는 에포크 수로 조정 가능
    verbose=1,
)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
Epoch 1/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 289s 1s/step - accuracy: 0.7036 - auc: 0.8970 - loss: 0.8689 - val_accuracy: 0.7308 - val_auc: 0.8205 - val_loss: 48.6358
Epoch 2/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 194s 953ms/step - accuracy: 0.7284 - auc: 0.9248 - loss: 0.6701 - val_accuracy: 0.0603 - val_auc: 0.5869 - val_loss: 1.8814
Epoch 3/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 182s 943ms/step - accuracy: 0.7437 - auc: 0.9293 - loss: 0.6476 - val_accuracy: 0.7308 - val_auc: 0.8348 - val_loss: 0.9749
Epoch 4/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 184s 956ms/step - accuracy: 0.7616 - auc: 0.9421 - loss: 0.5759 - val_accuracy: 0.7308 - val_auc: 0.8380 - val_loss: 2.7289
Epoch 5/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 184s 954ms/step - accuracy: 0.7773 - auc: 0.9433 - loss: 0.5763 - val_accuracy: 0.7314 - val_auc: 0.9025 - val_loss: 1.1371
Epoch 6/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 212s 990ms/step - accuracy: 0.7786 - auc: 0.9483 - loss: 0.5475 - val_accuracy: 0.7540 - va

KeyboardInterrupt: 

In [ ]:
# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test AUC: {test_auc}")

###DenseNet121

In [ ]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# DenseNet121 모델 정의 (ImageNet 사전 학습된 가중치 사용)
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모델을 쌓음
model = Sequential([
    base_model,  # DenseNet121 기반 모델
    GlobalAveragePooling2D(),  # 전역 평균 풀링
    Dense(128, activation='relu'),  # 중간 레이어
    Dense(len(top_4_classes), activation='softmax')  # 출력 레이어: 4개 클래스
])

# 모델 컴파일 (accuracy와 함께 AUC 추가)
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])  # AUC 추가

# 모델 학습
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,  # 원하는 에포크 수로 조정 가능
    verbose=1,
)


29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Epoch 1/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 529s 2s/step - accuracy: 0.7168 - auc: 0.9139 - loss: 0.7119 - val_accuracy: 0.7719 - val_auc: 0.9255 - val_loss: 1.0084
Epoch 2/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 178s 918ms/step - accuracy: 0.7745 - auc: 0.9402 - loss: 0.5893 - val_accuracy: 0.7434 - val_auc: 0.9304 - val_loss: 0.6801
Epoch 3/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 176s 909ms/step - accuracy: 0.7820 - auc: 0.9504 - loss: 0.5336 - val_accuracy: 0.7347 - val_auc: 0.9035 - val_loss: 0.8123
Epoch 4/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 176s 908ms/step - accuracy: 0.7875 - auc: 0.9491 - loss: 0.5463 - val_accuracy: 0.7865 - val_auc: 0.9484 - val_loss: 0.5669
Epoch 5/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 214s 970ms/step - accuracy: 0.7965 - auc: 0.9564 - loss: 0.5012 - val_accuracy: 0.7812 - val_auc: 0.9315 - val_loss: 0.6362
Epoch 6/10
189/189 ━━━━━━━━━━━━━━━━━━━━ 197s 1s/step - accuracy: 0.8030 - auc: 0.9566 - loss: 0.4993 - val_accuracy: 0.7692 - val_au

In [ ]:
# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test AUC: {test_auc}")

NameError: name 'model' is not defined

##3주차 - 데이터 필터링 + 촬영부위 라벨링 추가 (오류 못고침)


In [ ]:
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# 고유한 클래스 추출 및 클래스 개수 계산
class_counts = metadata_df['dx'].value_counts()  # 클래스별 데이터 개수 계산
top_4_classes = class_counts.head(4).index  # 상위 4개 클래스 추출

print(f"Top 4 classes: {top_4_classes}")

# 상위 4개 클래스만 포함한 데이터프레임 필터링
filtered_df = metadata_df[metadata_df['dx'].isin(top_4_classes)]

# 'image_path', 'localization', 'dx' 열만 남기고 나머지 드랍
filtered_df = filtered_df[['image_path', 'localization', 'dx']]

# 촬영 부위(localization) 열을 one-hot encoding
encoder = OneHotEncoder(sparse_output=False)  # sparse -> sparse_output으로 변경
localization_encoded = encoder.fit_transform(filtered_df[['localization']])

# Add the encoded localization to the DataFrame
localization_df = pd.DataFrame(localization_encoded, columns=encoder.categories_[0])
filtered_df = pd.concat([filtered_df, localization_df], axis=1)
filtered_df = filtered_df.dropna(subset=['image_path', 'localization', 'dx'])
# print(filtered_df.isna().sum())

# 메타데이터에서 데이터와 레이블 추출 (train/test로 변경)
train_df, test_df = train_test_split(filtered_df, test_size=0.2, stratify=filtered_df['dx'], random_state=42)

# 학습 데이터용 ImageDataGenerator (데이터 증강 + 학습/검증 분리)
train_datagen = ImageDataGenerator(
    rescale=1./255,                # 정규화
    rotation_range=40,             # 이미지를 랜덤하게 40도까지 회전
    width_shift_range=0.2,         # 이미지의 가로 위치를 최대 20%까지 이동
    height_shift_range=0.2,        # 이미지의 세로 위치를 최대 20%까지 이동
    shear_range=0.2,               # 랜덤한 전단 변환 (shear)
    zoom_range=0.2,                # 랜덤하게 이미지 크기를 20%까지 확대/축소
    horizontal_flip=True,          # 이미지를 좌우로 랜덤하게 반전
    fill_mode='nearest',           # 이미지 이동 시 생기는 빈 공간을 채우는 방식
    validation_split=0.2           # 학습 데이터 중 20%를 검증용으로 분리
)

# 테스트 데이터용 ImageDataGenerator (데이터 증강 없음, 정규화만 적용)
test_datagen = ImageDataGenerator(
    rescale=1./255  # 테스트 데이터는 증강 없이 정규화만 적용
)

# 학습 데이터 생성기 (이미지와 localization 정보를 함께 다룸)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,
    x_col='image_path',
    y_col='dx',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

# 검증 데이터 생성기 (이미지와 localization 정보를 함께 다룸)
val_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,
    x_col='image_path',
    y_col='dx',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

# 테스트 데이터 생성기 (이미지와 localization 정보를 함께 다룸)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=None,
    x_col='image_path',
    y_col='dx',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# 생성기 출력 테스트
print(f"Training data shape: {train_generator.image_shape}")
print(f"Validation data shape: {val_generator.image_shape}")
print(f"Test data shape: {test_generator.image_shape}")


Top 4 classes: Index(['nv', 'mel', 'bkl', 'bcc'], dtype='object', name='dx')
Found 6036 validated image filenames belonging to 4 classes.
Found 1508 validated image filenames belonging to 4 classes.
Found 1887 validated image filenames belonging to 4 classes.
Training data shape: (224, 224, 3)
Validation data shape: (224, 224, 3)
Test data shape: (224, 224, 3)


###모델 학습 + 클래스 불균형 해소를 위한 가중치 추가

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# 클래스별 샘플 수에 따른 가중치 계산
class_counts = filtered_df['dx'].value_counts()  # 각 클래스의 샘플 수
total_samples = len(filtered_df)  # 전체 샘플 수
class_weights = {i: total_samples / class_counts[cls] for i, cls in enumerate(top_4_classes)}

print(f"Class weights: {class_weights}")

# EfficientNetB0 모델 정의 (ImageNet 사전 학습된 가중치 사용)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모델을 쌓음
model = Sequential([
    base_model,  # EfficientNetB0 기반 모델
    GlobalAveragePooling2D(),  # 전역 평균 풀링
    Dense(128, activation='relu'),  # 중간 레이어
    Dense(len(top_4_classes), activation='softmax')  # 출력 레이어: 4개 클래스
])
# 모델 컴파일 (accuracy와 함께 AUC 추가)
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])  # AUC 추가

# 모델 학습
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,  # 원하는 에포크 수로 조정 가능
    verbose=1,
    class_weight=class_weights  # 클래스 가중치 추가
)


Class weights: {0: 1.4065622669649516, 1: 8.473495058400719, 2: 8.581437670609645, 3: 18.348249027237355}
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 80/189 ━━━━━━━━━━━━━━━━━━━━ 2:28 1s/step - accuracy: 0.6916 - auc: 0.8887 - loss: 7.7054

KeyboardInterrupt: 

In [ ]:
# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test AUC: {test_auc}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 30s 478ms/step - accuracy: 0.7928 - auc: 0.9532 - loss: 0.8157
Test accuracy: 0.7973040342330933
Test AUC: 0.9569269418716431


###모델 학습. 클래스 불균형을 위한 가중치 추가 없음

In [ ]:
# EfficientNetB0 모델 정의 (ImageNet 사전 학습된 가중치 사용)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모델을 쌓음
model = Sequential([
    base_model,  # EfficientNetB0 기반 모델
    GlobalAveragePooling2D(),  # 전역 평균 풀링
    Dense(128, activation='relu'),  # 중간 레이어
    Dense(len(top_4_classes), activation='softmax')  # 출력 레이어: 4개 클래스
])
# 모델 컴파일 (accuracy와 함께 AUC 추가)
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])  # AUC 추가

# 모델 학습
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,  # 원하는 에포크 수로 조정 가능
    verbose=1,
    class_weight=class_weights  # 클래스 가중치 추가
)


##4주차 - 대학생 논문 기반 테스트

###데이터 로드 + 필터링 안함


In [ ]:
# Kaggle API 설치
!pip install kaggle

# Kaggle API Key 설정
import json
import os
from google.colab import files
files.upload()  # kaggle.json 업로드

# Kaggle 설정
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 데이터셋 다운로드 (HAM10000 데이터셋)
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000

# 압축 해제
!unzip skin-cancer-mnist-ham10000.zip


Saving kaggle (1).json to kaggle (1).json
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/kmader/skin-cancer-mnist-ham10000
License(s): CC-BY-NC-SA-4.0
  5% 261M/5.20G [00:13<03:28, 25.4MB/s]

In [ ]:
import os
import pandas as pd

# CSV 파일을 불러와서 DataFrame으로 저장
metadata_df = pd.read_csv('./HAM10000_metadata.csv')

# 이미지 폴더 경로
image_folder_1 = './HAM10000_images_part_1/'
image_folder_2 = './HAM10000_images_part_2/'

# 이미지 파일 경로 생성
metadata_df['image_path'] = metadata_df['image_id'].apply(
    lambda x: os.path.join(image_folder_1, f"{x}.jpg")
              if os.path.exists(os.path.join(image_folder_1, f"{x}.jpg"))
              else os.path.join(image_folder_2, f"{x}.jpg"))

# 메타데이터에 이미지 경로 추가 확인
print(metadata_df[['image_id', 'image_path']].head())


In [ ]:
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# 고유한 클래스 추출 및 클래스 개수 계산
unique_classes = metadata_df['dx'].unique()
NUM_CLASSES = len(unique_classes)

print(f"Number of classes: {NUM_CLASSES}")
print(f"Classes: {unique_classes}")

# 메타데이터에서 데이터와 레이블 추출 (train/test로 변경)
train_df, test_df = train_test_split(metadata_df, test_size=0.2, stratify=metadata_df['dx'], random_state=42)


# 학습 데이터용 ImageDataGenerator (데이터 증강 + 학습/검증 분리)
train_datagen = ImageDataGenerator(
    rescale=1./255,                # 정규화
    rotation_range=40,             # 이미지를 랜덤하게 40도까지 회전
    width_shift_range=0.2,         # 이미지의 가로 위치를 최대 20%까지 이동
    height_shift_range=0.2,        # 이미지의 세로 위치를 최대 20%까지 이동
    shear_range=0.2,               # 랜덤한 전단 변환 (shear)
    zoom_range=0.2,                # 랜덤하게 이미지 크기를 20%까지 확대/축소
    horizontal_flip=True,          # 이미지를 좌우로 랜덤하게 반전
    fill_mode='nearest',           # 이미지 이동 시 생기는 빈 공간을 채우는 방식
    validation_split=0.2           # 학습 데이터 중 20%를 검증용으로 분리
)

# 테스트 데이터용 ImageDataGenerator (데이터 증강 없음, 정규화만 적용)
test_datagen = ImageDataGenerator(
    rescale=1./255  # 테스트 데이터는 증강 없이 정규화만 적용
)

# 학습 데이터 생성기 (학습용)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,
    x_col='image_path',
    y_col='dx',
    target_size=(456, 456),
    batch_size=8,
    class_mode='categorical',
    subset='training',  # 학습용 데이터
    shuffle=True
)

# 학습 데이터 생성기 (검증용)
val_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,
    x_col='image_path',
    y_col='dx',
    target_size=(456, 456),
    batch_size=8,
    class_mode='categorical',
    subset='validation',  # 검증용 데이터
    shuffle=True
)

# 테스트 데이터 생성기
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=None,
    x_col='image_path',
    y_col='dx',
    target_size=(456, 456),
    batch_size=8,
    class_mode='categorical',
    shuffle=False  # 테스트 데이터는 shuffle하지 않음
)

# 생성기 출력 테스트
print(f"Training data shape: {train_generator.image_shape}")
print(f"Validation data shape: {val_generator.image_shape}")
print(f"Test data shape: {test_generator.image_shape}")

Number of classes: 7
Classes: ['bkl' 'nv' 'df' 'mel' 'vasc' 'bcc' 'akiec']
Found 6410 validated image filenames belonging to 7 classes.
Found 1602 validated image filenames belonging to 7 classes.
Found 2003 validated image filenames belonging to 7 classes.
Training data shape: (224, 224, 3)
Validation data shape: (224, 224, 3)
Test data shape: (224, 224, 3)


###모델 학습

In [ ]:
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# EfficientNetB5 모델 정의 (ImageNet 사전 학습된 가중치 사용)
base_model = EfficientNetB5(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모델을 쌓음
model = Sequential([
    base_model,  # EfficientNetB5 기반 모델
    GlobalAveragePooling2D(),  # 전역 평균 풀링
    Dense(128, activation='relu'),  # 중간 레이어
    Dense(NUM_CLASSES, activation='softmax')  # 출력 레이어: 클래스 수만큼 softmax
])

# 모델 컴파일 (accuracy와 함께 AUC 추가)
model.compile(optimizer=AdamW(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])  # AUC 추가

early_stopping = EarlyStopping(
    monitor='val_loss',  # 검증 데이터의 loss를 기준으로 모니터링
    patience=5,          # 5 epoch 동안 성능 향상이 없으면 중단
    restore_best_weights=True  # 가장 성능이 좋은 epoch의 가중치를 복원
)

# 모델 학습
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,  # 최대 20 epoch 동안 학습 (EarlyStopping이 적용되면 더 일찍 종료 가능)
    verbose=1,
    callbacks=[early_stopping]  # EarlyStopping 콜백 추가
)


Epoch 1/20
802/802 ━━━━━━━━━━━━━━━━━━━━ 767s 620ms/step - accuracy: 0.6637 - auc: 0.9113 - loss: 0.9904 - val_accuracy: 0.7703 - val_auc: 0.9609 - val_loss: 0.6786
Epoch 2/20
802/802 ━━━━━━━━━━━━━━━━━━━━ 289s 359ms/step - accuracy: 0.7830 - auc: 0.9701 - loss: 0.5931 - val_accuracy: 0.8327 - val_auc: 0.9827 - val_loss: 0.4560
Epoch 3/20
802/802 ━━━━━━━━━━━━━━━━━━━━ 345s 387ms/step - accuracy: 0.8074 - auc: 0.9767 - loss: 0.5194 - val_accuracy: 0.8521 - val_auc: 0.9842 - val_loss: 0.4280
Epoch 4/20
802/802 ━━━━━━━━━━━━━━━━━━━━ 314s 391ms/step - accuracy: 0.8537 - auc: 0.9861 - loss: 0.3954 - val_accuracy: 0.8421 - val_auc: 0.9823 - val_loss: 0.4373
Epoch 5/20
802/802 ━━━━━━━━━━━━━━━━━━━━ 294s 355ms/step - accuracy: 0.8764 - auc: 0.9884 - loss: 0.3499 - val_accuracy: 0.8552 - val_auc: 0.9848 - val_loss: 0.4173
Epoch 6/20
802/802 ━━━━━━━━━━━━━━━━━━━━ 349s 390ms/step - accuracy: 0.8849 - auc: 0.9905 - loss: 0.3204 - val_accuracy: 0.8596 - val_auc: 0.9830 - val_loss: 0.4226
Epoch 7/20
802/8

In [ ]:
# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test AUC: {test_auc}")

251/251 ━━━━━━━━━━━━━━━━━━━━ 55s 221ms/step - accuracy: 0.8182 - auc: 0.9770 - loss: 0.5174
Test accuracy: 0.8277583718299866
Test AUC: 0.9784810543060303


##5주차 - 대학생 논문 기반 테스트 입력값 및 에포크 수정 (456,456) 15 , 가중치 복원 및 스케줄러 추가, 배치사이즈 16 수정

###데이터 로드 + 필터링 안함


In [ ]:
# Kaggle API 설치
!pip install kaggle

# Kaggle API Key 설정
import json
import os
from google.colab import files
files.upload()  # kaggle.json 업로드

# Kaggle 설정
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 데이터셋 다운로드 (HAM10000 데이터셋)
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000

# 압축 해제
!unzip skin-cancer-mnist-ham10000.zip


Saving kaggle (1).json to kaggle (1) (1).json
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/kmader/skin-cancer-mnist-ham10000
License(s): CC-BY-NC-SA-4.0
skin-cancer-mnist-ham10000.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  skin-cancer-mnist-ham10000.zip
replace HAM10000_images_part_1/ISIC_0024306.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
import os
import pandas as pd

# CSV 파일을 불러와서 DataFrame으로 저장
metadata_df = pd.read_csv('./HAM10000_metadata.csv')

# 이미지 폴더 경로
image_folder_1 = './HAM10000_images_part_1/'
image_folder_2 = './HAM10000_images_part_2/'

# 이미지 파일 경로 생성
metadata_df['image_path'] = metadata_df['image_id'].apply(
    lambda x: os.path.join(image_folder_1, f"{x}.jpg")
              if os.path.exists(os.path.join(image_folder_1, f"{x}.jpg"))
              else os.path.join(image_folder_2, f"{x}.jpg"))

# 메타데이터에 이미지 경로 추가 확인
print(metadata_df[['image_id', 'image_path']].head())


       image_id                                 image_path
0  ISIC_0027419  ./HAM10000_images_part_1/ISIC_0027419.jpg
1  ISIC_0025030  ./HAM10000_images_part_1/ISIC_0025030.jpg
2  ISIC_0026769  ./HAM10000_images_part_1/ISIC_0026769.jpg
3  ISIC_0025661  ./HAM10000_images_part_1/ISIC_0025661.jpg
4  ISIC_0031633  ./HAM10000_images_part_2/ISIC_0031633.jpg


In [ ]:
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# 고유한 클래스 추출 및 클래스 개수 계산
unique_classes = metadata_df['dx'].unique()
NUM_CLASSES = len(unique_classes)

print(f"Number of classes: {NUM_CLASSES}")
print(f"Classes: {unique_classes}")

# 메타데이터에서 데이터와 레이블 추출 (train/test로 변경)
train_df, test_df = train_test_split(metadata_df, test_size=0.2, stratify=metadata_df['dx'], random_state=42)


# 학습 데이터용 ImageDataGenerator (데이터 증강 + 학습/검증 분리)
train_datagen = ImageDataGenerator(
    rescale=1./255,                # 정규화
    rotation_range=40,             # 이미지를 랜덤하게 40도까지 회전
    width_shift_range=0.2,         # 이미지의 가로 위치를 최대 20%까지 이동
    height_shift_range=0.2,        # 이미지의 세로 위치를 최대 20%까지 이동
    shear_range=0.2,               # 랜덤한 전단 변환 (shear)
    zoom_range=0.2,                # 랜덤하게 이미지 크기를 20%까지 확대/축소
    horizontal_flip=True,          # 이미지를 좌우로 랜덤하게 반전
    fill_mode='nearest',           # 이미지 이동 시 생기는 빈 공간을 채우는 방식
    validation_split=0.2           # 학습 데이터 중 20%를 검증용으로 분리
)

# 테스트 데이터용 ImageDataGenerator (데이터 증강 없음, 정규화만 적용)
test_datagen = ImageDataGenerator(
    rescale=1./255  # 테스트 데이터는 증강 없이 정규화만 적용
)

# 학습 데이터 생성기 (학습용)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,
    x_col='image_path',
    y_col='dx',
    target_size=(456, 456),
    batch_size=8,
    class_mode='categorical',
    subset='training',  # 학습용 데이터
    shuffle=True
)

# 학습 데이터 생성기 (검증용)
val_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,
    x_col='image_path',
    y_col='dx',
    target_size=(456, 456),
    batch_size=8,
    class_mode='categorical',
    subset='validation',  # 검증용 데이터
    shuffle=True
)

# 테스트 데이터 생성기
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=None,
    x_col='image_path',
    y_col='dx',
    target_size=(456, 456),
    batch_size=8,
    class_mode='categorical',
    shuffle=False  # 테스트 데이터는 shuffle하지 않음
)

# 생성기 출력 테스트
print(f"Training data shape: {train_generator.image_shape}")
print(f"Validation data shape: {val_generator.image_shape}")
print(f"Test data shape: {test_generator.image_shape}")

Number of classes: 7
Classes: ['bkl' 'nv' 'df' 'mel' 'vasc' 'bcc' 'akiec']
Found 6410 validated image filenames belonging to 7 classes.
Found 1602 validated image filenames belonging to 7 classes.
Found 2003 validated image filenames belonging to 7 classes.
Training data shape: (456, 456, 3)
Validation data shape: (456, 456, 3)
Test data shape: (456, 456, 3)


###모델 학습

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import ReduceLROnPlateau
import tensorflow as tf

# EfficientNetB5 모델 정의 (ImageNet 사전 학습된 가중치 사용)
base_model = EfficientNetB5(weights='imagenet', include_top=False, input_shape=(456, 456, 3))

# AdamW 옵티마이저 설정
optimizer = AdamW(
    learning_rate=0.0001,  # 학습률
    weight_decay=1e-4,     # 가중치 감쇠
    beta_1=0.9,            # 1차 모멘텀 감쇠율
    beta_2=0.999,          # 2차 모멘텀 감쇠율
    epsilon=1e-7           # 작은 값 (0으로 나누는 문제 방지)
)

# ReduceLROnPlateau 콜백 (성능 향상이 없을 때 학습률을 줄임)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',  # val_loss를 모니터링하여 학습률을 조절
    factor=0.5,          # 학습률을 절반으로 줄임
    patience=3,          # 3 epoch 동안 성능 향상이 없으면 학습률을 줄임
    min_lr=1e-6          # 학습률이 1e-6 이하로 떨어지지 않도록 설정
)

# ModelCheckpoint 콜백 (가장 좋은 가중치를 저장)
checkpoint = ModelCheckpoint(
    'best_model.weights.h5',    # 저장할 파일 이름
    monitor='val_loss',  # 검증 손실을 기준으로 모니터링
    save_best_only=True, # 가장 좋은 가중치만 저장
    mode='min',          # 검증 손실이 최소일 때 가중치 저장
    save_weights_only=True  # 가중치만 저장
)

# 모델을 쌓음
model = Sequential([
    base_model,  # EfficientNetB5 기반 모델
    GlobalAveragePooling2D(),  # 전역 평균 풀링
    Dense(128, activation='relu'),  # 중간 레이어
    Dense(NUM_CLASSES, activation='softmax')  # 출력 레이어: 클래스 수만큼 softmax
])

# 모델 컴파일 (accuracy와 함께 AUC 추가)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

# 모델 학습
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,
    verbose=1,
    callbacks=[reduce_lr, checkpoint]  # ReduceLROnPlateau와 ModelCheckpoint 추가
)

# 가장 좋은 가중치 복원
model.load_weights('best_model.weights.h5')

115263384/115263384 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


802/802 ━━━━━━━━━━━━━━━━━━━━ 1050s 1s/step - accuracy: 0.6775 - auc: 0.9147 - loss: 0.9566 - val_accuracy: 0.7834 - val_auc: 0.9663 - val_loss: 0.6328 - learning_rate: 1.0000e-04
Epoch 2/15
802/802 ━━━━━━━━━━━━━━━━━━━━ 698s 818ms/step - accuracy: 0.8193 - auc: 0.9786 - loss: 0.4898 - val_accuracy: 0.7965 - val_auc: 0.9759 - val_loss: 0.5259 - learning_rate: 1.0000e-04
Epoch 3/15
802/802 ━━━━━━━━━━━━━━━━━━━━ 678s 817ms/step - accuracy: 0.8694 - auc: 0.9883 - loss: 0.3631 - val_accuracy: 0.8614 - val_auc: 0.9842 - val_loss: 0.4019 - learning_rate: 1.0000e-04
Epoch 4/15
802/802 ━━━━━━━━━━━━━━━━━━━━ 716s 857ms/step - accuracy: 0.8921 - auc: 0.9918 - loss: 0.2958 - val_accuracy: 0.8614 - val_auc: 0.9831 - val_loss: 0.4205 - learning_rate: 1.0000e-04
Epoch 5/15
802/802 ━━━━━━━━━━━━━━━━━━━━ 650s 807ms/step - accuracy: 0.9165 - auc: 0.9943 - loss: 0.2369 - val_accuracy: 0.8814 - val_auc: 0.9866 - val_loss: 0.3734 - learning_rate: 1.0000e-04
Epoch 6/15
802/802 ━━━━━━━━━━━━━━━━━━━━ 635s 749ms/st

KeyboardInterrupt: 

In [ ]:
# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")
print(f"Test AUC: {test_auc}")

251/251 ━━━━━━━━━━━━━━━━━━━━ 39s 154ms/step - accuracy: 0.8580 - auc: 0.9750 - loss: 0.4937
Test accuracy: 0.8647029399871826
Test loss: 0.48421555757522583
Test AUC: 0.9765500426292419


In [ ]:
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf

# 클래스 수 정의 (현재 클래스 수 설정)
NUM_CLASSES = 7

# EfficientNetB5 모델 구조 정의 (이전과 동일한 구조로 정의)
model = Sequential([
    EfficientNetB5(weights=None, include_top=False, input_shape=(456, 456, 3)),
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

# 옵티마이저 설정 (이전과 동일하게 설정)
optimizer = AdamW(
    learning_rate=0.0001,
    weight_decay=1e-4,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-7
)

# ReduceLROnPlateau 콜백 (성능 향상이 없을 때 학습률을 줄임)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',  # val_loss를 모니터링하여 학습률을 조절
    factor=0.5,          # 학습률을 절반으로 줄임
    patience=3,          # 3 epoch 동안 성능 향상이 없으면 학습률을 줄임
    min_lr=1e-6          # 학습률이 1e-6 이하로 떨어지지 않도록 설정
)

# ModelCheckpoint 콜백 (가장 좋은 가중치를 저장)
checkpoint = ModelCheckpoint(
    'best_model.weights.h5',    # 저장할 파일 이름
    monitor='val_loss',  # 검증 손실을 기준으로 모니터링
    save_best_only=True, # 가장 좋은 가중치만 저장
    mode='min',          # 검증 손실이 최소일 때 가중치 저장
    save_weights_only=True  # 가중치만 저장
)

# 모델 컴파일
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

# 모델을 명시적으로 빌드하여 레이어들이 초기화되도록 함
sample_input = tf.random.normal([1, 456, 456, 3])  # 샘플 입력으로 모델 빌드
model(sample_input)  # 이 라인에서 모델이 빌드됨

# 이전 학습에서 저장된 가중치를 복원 (모델이 빌드된 후에 실행)
model.load_weights('best_model.weights.h5')

# 모델을 복원한 후 학습 이어가기
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,  # 추가 학습할 에포크 수
    verbose=1,
    callbacks=[reduce_lr, checkpoint]  # 필요한 콜백 다시 설정
)


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adamw', because it has 2 variables whereas the saved optimizer has 1018 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


802/802 ━━━━━━━━━━━━━━━━━━━━ 1063s 1s/step - accuracy: 0.9162 - auc: 0.9948 - loss: 0.2297 - val_accuracy: 0.6810 - val_auc: 0.8608 - val_loss: 2.1021 - learning_rate: 1.0000e-04
Epoch 2/5
802/802 ━━━━━━━━━━━━━━━━━━━━ 703s 838ms/step - accuracy: 0.9527 - auc: 0.9967 - loss: 0.1452 - val_accuracy: 0.7846 - val_auc: 0.9435 - val_loss: 0.9631 - learning_rate: 1.0000e-04
Epoch 3/5
802/802 ━━━━━━━━━━━━━━━━━━━━ 651s 809ms/step - accuracy: 0.9554 - auc: 0.9979 - loss: 0.1264 - val_accuracy: 0.8452 - val_auc: 0.9725 - val_loss: 0.6043 - learning_rate: 1.0000e-04
Epoch 4/5
 75/802 ━━━━━━━━━━━━━━━━━━━━ 7:54 652ms/step - accuracy: 0.9485 - auc: 0.9986 - loss: 0.1337

KeyboardInterrupt: 

In [ ]:
# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")
print(f"Test AUC: {test_auc}")

251/251 ━━━━━━━━━━━━━━━━━━━━ 56s 221ms/step - accuracy: 0.6768 - auc: 0.8644 - loss: 2.4485
Test accuracy: 0.6889665722846985
Test loss: 2.398390769958496
Test AUC: 0.8657111525535583


In [ ]:
model.load_weights('best_model.weights.h5')

##9주차 - 연속학습을 위해 5주차에 가중치, 옵티마이저, 스케줄러 업데이트 저장 추가

###데이터 로드 + 필터링 안함


In [ ]:
# Kaggle API 설치
!pip install kaggle

# Kaggle API Key 설정
import json
import os
from google.colab import files
files.upload()  # kaggle.json 업로드

# Kaggle 설정
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 데이터셋 다운로드 (HAM10000 데이터셋)
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000

# 압축 해제
!unzip skin-cancer-mnist-ham10000.zip


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: ham10000_images_part_2/ISIC_0029325.jpg  
  inflating: ham10000_images_part_2/ISIC_0029326.jpg  
  inflating: ham10000_images_part_2/ISIC_0029327.jpg  
  inflating: ham10000_images_part_2/ISIC_0029328.jpg  
  inflating: ham10000_images_part_2/ISIC_0029329.jpg  
  inflating: ham10000_images_part_2/ISIC_0029330.jpg  
  inflating: ham10000_images_part_2/ISIC_0029331.jpg  
  inflating: ham10000_images_part_2/ISIC_0029332.jpg  
  inflating: ham10000_images_part_2/ISIC_0029333.jpg  
  inflating: ham10000_images_part_2/ISIC_0029334.jpg  
  inflating: ham10000_images_part_2/ISIC_0029335.jpg  
  inflating: ham10000_images_part_2/ISIC_0029336.jpg  
  inflating: ham10000_images_part_2/ISIC_0029337.jpg  
  inflating: ham10000_images_part_2/ISIC_0029338.jpg  
  inflating: ham10000_images_part_2/ISIC_0029339.jpg  
  inflating: ham10000_images_part_2/ISIC_0029340.jpg  
  inflating: ham10000_images_part_2/ISIC_0029341.jpg  
  inflating: ham10000_images_

In [ ]:
import os
import pandas as pd

# CSV 파일을 불러와서 DataFrame으로 저장
metadata_df = pd.read_csv('./HAM10000_metadata.csv')

# 이미지 폴더 경로
image_folder_1 = './HAM10000_images_part_1/'
image_folder_2 = './HAM10000_images_part_2/'

# 이미지 파일 경로 생성
metadata_df['image_path'] = metadata_df['image_id'].apply(
    lambda x: os.path.join(image_folder_1, f"{x}.jpg")
              if os.path.exists(os.path.join(image_folder_1, f"{x}.jpg"))
              else os.path.join(image_folder_2, f"{x}.jpg"))

# 메타데이터에 이미지 경로 추가 확인
print(metadata_df[['image_id', 'image_path']].head())


       image_id                                 image_path
0  ISIC_0027419  ./HAM10000_images_part_1/ISIC_0027419.jpg
1  ISIC_0025030  ./HAM10000_images_part_1/ISIC_0025030.jpg
2  ISIC_0026769  ./HAM10000_images_part_1/ISIC_0026769.jpg
3  ISIC_0025661  ./HAM10000_images_part_1/ISIC_0025661.jpg
4  ISIC_0031633  ./HAM10000_images_part_2/ISIC_0031633.jpg


In [ ]:
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# 고유한 클래스 추출 및 클래스 개수 계산
unique_classes = metadata_df['dx'].unique()
NUM_CLASSES = len(unique_classes)

print(f"Number of classes: {NUM_CLASSES}")
print(f"Classes: {unique_classes}")

# 메타데이터에서 데이터와 레이블 추출 (train/test로 변경)
train_df, test_df = train_test_split(metadata_df, test_size=0.2, stratify=metadata_df['dx'], random_state=42)


# 학습 데이터용 ImageDataGenerator (데이터 증강 + 학습/검증 분리)
train_datagen = ImageDataGenerator(
    rescale=1./255,                # 정규화
    rotation_range=40,             # 이미지를 랜덤하게 40도까지 회전
    width_shift_range=0.2,         # 이미지의 가로 위치를 최대 20%까지 이동
    height_shift_range=0.2,        # 이미지의 세로 위치를 최대 20%까지 이동
    shear_range=0.2,               # 랜덤한 전단 변환 (shear)
    zoom_range=0.2,                # 랜덤하게 이미지 크기를 20%까지 확대/축소
    horizontal_flip=True,          # 이미지를 좌우로 랜덤하게 반전
    fill_mode='nearest',           # 이미지 이동 시 생기는 빈 공간을 채우는 방식
    validation_split=0.2           # 학습 데이터 중 20%를 검증용으로 분리
)

# 테스트 데이터용 ImageDataGenerator (데이터 증강 없음, 정규화만 적용)
test_datagen = ImageDataGenerator(
    rescale=1./255  # 테스트 데이터는 증강 없이 정규화만 적용
)

# 학습 데이터 생성기 (학습용)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,
    x_col='image_path',
    y_col='dx',
    target_size=(456, 456),
    batch_size=8,
    class_mode='categorical',
    subset='training',  # 학습용 데이터
    shuffle=True
)

# 학습 데이터 생성기 (검증용)
val_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,
    x_col='image_path',
    y_col='dx',
    target_size=(456, 456),
    batch_size=8,
    class_mode='categorical',
    subset='validation',  # 검증용 데이터
    shuffle=True
)

# 테스트 데이터 생성기
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=None,
    x_col='image_path',
    y_col='dx',
    target_size=(456, 456),
    batch_size=8,
    class_mode='categorical',
    shuffle=False  # 테스트 데이터는 shuffle하지 않음
)

# 생성기 출력 테스트
print(f"Training data shape: {train_generator.image_shape}")
print(f"Validation data shape: {val_generator.image_shape}")
print(f"Test data shape: {test_generator.image_shape}")

Number of classes: 7
Classes: ['bkl' 'nv' 'df' 'mel' 'vasc' 'bcc' 'akiec']
Found 6410 validated image filenames belonging to 7 classes.
Found 1602 validated image filenames belonging to 7 classes.
Found 2003 validated image filenames belonging to 7 classes.
Training data shape: (456, 456, 3)
Validation data shape: (456, 456, 3)
Test data shape: (456, 456, 3)


###모델 학습 - 1차

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import ReduceLROnPlateau
import json

# 클래스 수 정의
NUM_CLASSES = 7

# EfficientNetB5 모델 정의
base_model = EfficientNetB5(weights='imagenet', include_top=False, input_shape=(456, 456, 3))

# 모델 설정
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

# 옵티마이저 설정
optimizer = AdamW(
    learning_rate=0.0001,
    weight_decay=1e-4,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-7
)

# ReduceLROnPlateau 스케줄러 설정
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6
)

# 모델 컴파일
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

# 학습률 저장 함수
def save_learning_rate(optimizer, filename='learning_rate.txt'):
    with open(filename, 'w') as f:
        f.write(str(optimizer.learning_rate.numpy()))
    print("현재 학습률이 저장되었습니다.")

# 학습률 복원 함수
def load_learning_rate(optimizer, filename='learning_rate.txt'):
    try:
        with open(filename, 'r') as f:
            learning_rate = float(f.read())
            optimizer.learning_rate.assign(learning_rate)
        print(f"학습률이 {learning_rate}로 복원되었습니다.")
    except FileNotFoundError:
        print("학습률 파일을 찾을 수 없습니다. 초기 학습률로 시작합니다.")

# 체크포인트와 최적 검증 손실을 위한 변수 설정
best_checkpoint_path = 'best_model_checkpoint'
last_checkpoint_path = 'last_model_checkpoint'

best_checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)
last_checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)

best_manager = tf.train.CheckpointManager(best_checkpoint, best_checkpoint_path, max_to_keep=1)
last_manager = tf.train.CheckpointManager(last_checkpoint, last_checkpoint_path, max_to_keep=1)

# 이전 학습 상태 복원
### 테스트 결과에 따라 latest_checkpoint인지 best_checkpoint인지 결정 예정
best_val_loss = float('inf')  # 최적의 검증 손실 추적
if best_manager.latest_checkpoint:
    best_checkpoint.restore(best_manager.latest_checkpoint)
    load_learning_rate(optimizer)  # 학습률 복원
    print("가장 최적의 상태로 복원이 완료되었습니다.")
else:
    print("저장된 체크포인트가 없습니다. 새로 학습을 시작합니다.")

# 사용자 정의 콜백으로 최적 상태 저장
class CustomCheckpointCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        global best_val_loss
        val_loss = logs.get('val_loss')

        # 최적 검증 손실 갱신 시 최적 상태 저장
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_manager.save()  # 최적 상태 저장
            save_learning_rate(optimizer)  # 학습률 저장
            print(f"Best checkpoint saved at epoch {epoch+1} with val_loss {val_loss:.4f}")

        # 매 에포크 종료 시마다 마지막 학습 상태 저장
        last_manager.save()
        print(f"Last checkpoint saved at epoch {epoch+1}")

# 모델 학습
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=8,
    verbose=1,
    callbacks=[reduce_lr, CustomCheckpointCallback()]  # 사용자 정의 체크포인트 콜백 추가
)


115263384/115263384 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
저장된 체크포인트가 없습니다. 새로 학습을 시작합니다.
Epoch 1/8


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


802/802 ━━━━━━━━━━━━━━━━━━━━ 0s 853ms/step - accuracy: 0.6872 - auc: 0.9212 - loss: 0.9263현재 학습률이 저장되었습니다.
Best checkpoint saved at epoch 1 with val_loss 0.5749
Last checkpoint saved at epoch 1
802/802 ━━━━━━━━━━━━━━━━━━━━ 1110s 1s/step - accuracy: 0.6873 - auc: 0.9213 - loss: 0.9261 - val_accuracy: 0.7953 - val_auc: 0.9716 - val_loss: 0.5749 - learning_rate: 1.0000e-04
Epoch 2/8
802/802 ━━━━━━━━━━━━━━━━━━━━ 0s 691ms/step - accuracy: 0.8222 - auc: 0.9779 - loss: 0.4983현재 학습률이 저장되었습니다.
Best checkpoint saved at epoch 2 with val_loss 0.4610
Last checkpoint saved at epoch 2
802/802 ━━━━━━━━━━━━━━━━━━━━ 675s 836ms/step - accuracy: 0.8222 - auc: 0.9779 - loss: 0.4983 - val_accuracy: 0.8352 - val_auc: 0.9810 - val_loss: 0.4610 - learning_rate: 1.0000e-04
Epoch 3/8
802/802 ━━━━━━━━━━━━━━━━━━━━ 0s 681ms/step - accuracy: 0.8629 - auc: 0.9866 - loss: 0.3843현재 학습률이 저장되었습니다.
Best checkpoint saved at epoch 3 with val_loss 0.3867
Last checkpoint saved at epoch 3
802/802 ━━━━━━━━━━━━━━━━━━━━ 670s 831m

In [ ]:
# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")
print(f"Test AUC: {test_auc}")

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


251/251 ━━━━━━━━━━━━━━━━━━━━ 61s 242ms/step - accuracy: 0.8596 - auc: 0.9794 - loss: 0.4826
Test accuracy: 0.8652021884918213
Test loss: 0.4607871472835541
Test AUC: 0.9802460074424744


In [ ]:
# 학습 종료 후 최적의 가중치로 모델 복원
if best_manager.latest_checkpoint:
    best_checkpoint.restore(best_manager.latest_checkpoint)
    print("학습 종료 후 최적의 가중치로 모델이 복원되었습니다.")
else:
    print("최적의 가중치를 찾을 수 없습니다.")


# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")
print(f"Test AUC: {test_auc}")

학습 종료 후 최적의 가중치로 모델이 복원되었습니다.
251/251 ━━━━━━━━━━━━━━━━━━━━ 35s 138ms/step - accuracy: 0.7933 - auc: 0.9715 - loss: 0.5968
Test accuracy: 0.8057913184165955
Test loss: 0.5699560046195984
Test AUC: 0.9731468558311462


In [ ]:
# 학습 종료 후 마지막 가중치로 모델 복원
if last_manager.latest_checkpoint:
    last_checkpoint.restore(last_manager.latest_checkpoint)
    print("학습 종료 후 마지막 가중치로 모델이 복원되었습니다.")
else:
    print("마지막 가중치를 찾을 수 없습니다.")

# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")
print(f"Test AUC: {test_auc}")

학습 종료 후 마지막 가중치로 모델이 복원되었습니다.
251/251 ━━━━━━━━━━━━━━━━━━━━ 35s 139ms/step - accuracy: 0.8596 - auc: 0.9794 - loss: 0.4826
Test accuracy: 0.8652021884918213
Test loss: 0.4607871472835541
Test AUC: 0.9802460074424744


###모델학습 - 2차

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import ReduceLROnPlateau
import json

# 클래스 수 정의
NUM_CLASSES = 7

# EfficientNetB5 모델 정의
base_model = EfficientNetB5(weights='imagenet', include_top=False, input_shape=(456, 456, 3))

# 모델 설정
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

# 옵티마이저 설정
optimizer = AdamW(
    learning_rate=0.0001,
    weight_decay=1e-4,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-7
)

# ReduceLROnPlateau 스케줄러 설정
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6
)

# 모델 컴파일
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

# 학습률 저장 함수 (마지막 학습률과 최소 손실 학습률을 한 파일에 저장)
def save_learning_rates(last_lr, best_lr, filename='learning_rates.txt'):
    with open(filename, 'w') as f:
        f.write(f"last_learning_rate: {last_lr}\n")
        f.write(f"best_learning_rate: {best_lr}\n")
    print("마지막 학습률과 최소 손실 학습률이 저장되었습니다.")


# 체크포인트와 최적 검증 손실을 위한 변수 설정
best_checkpoint_path = 'best_model_checkpoint/'
last_checkpoint_path = 'last_model_checkpoint/'

best_checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)
last_checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)

best_manager = tf.train.CheckpointManager(best_checkpoint, best_checkpoint_path, max_to_keep=1)
last_manager = tf.train.CheckpointManager(last_checkpoint, last_checkpoint_path, max_to_keep=1)


# 마지막 학습 상태 복원
if last_manager.latest_checkpoint:
    last_checkpoint.restore(last_manager.latest_checkpoint)
    print("마지막 학습 상태로 복원이 완료되었습니다.")
else:
    print("저장된 체크포인트가 없습니다. 새로 학습을 시작합니다.")

115263384/115263384 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
마지막 학습 상태로 복원이 완료되었습니다.


In [ ]:
# 사용자 정의 콜백으로 최적 상태 저장
class CustomCheckpointCallback(tf.keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
        self.best_val_loss = float('inf')
        self.best_lr = 0.0001

    def on_epoch_end(self, epoch, logs=None):
        val_loss = logs.get('val_loss')
        current_lr = float(self.model.optimizer.learning_rate.numpy())

        # 최적 검증 손실 갱신 시 최적 상태와 학습률 저장
        if val_loss < self.best_val_loss:
            self.best_val_loss = val_loss
            self.best_lr = current_lr
            best_manager.save()  # 최적 상태 저장
            print(f"Best checkpoint saved at epoch {epoch+1} with val_loss {val_loss:.4f}")

        # 마지막 학습 상태 저장
        last_manager.save()

        # 마지막 학습률과 최소 손실 학습률 저장
        save_learning_rates(current_lr, self.best_lr or current_lr)
        print(f"Last checkpoint saved at epoch {epoch+1} with last learning rate {current_lr}")

# 모델 학습
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    verbose=1,
    callbacks=[reduce_lr, CustomCheckpointCallback()]  # 사용자 정의 체크포인트 콜백 추가
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


802/802 ━━━━━━━━━━━━━━━━━━━━ 0s 881ms/step - accuracy: 0.9672 - auc: 0.9982 - loss: 0.0989Best checkpoint saved at epoch 1 with val_loss 0.3821
마지막 학습률과 최소 손실 학습률이 저장되었습니다.
Last checkpoint saved at epoch 1 with last learning rate 4.999999873689376e-05
802/802 ━━━━━━━━━━━━━━━━━━━━ 1113s 1s/step - accuracy: 0.9672 - auc: 0.9982 - loss: 0.0989 - val_accuracy: 0.8908 - val_auc: 0.9837 - val_loss: 0.3821 - learning_rate: 5.0000e-05
Epoch 2/10
802/802 ━━━━━━━━━━━━━━━━━━━━ 0s 731ms/step - accuracy: 0.9781 - auc: 0.9996 - loss: 0.0597Best checkpoint saved at epoch 2 with val_loss 0.3591
마지막 학습률과 최소 손실 학습률이 저장되었습니다.
Last checkpoint saved at epoch 2 with last learning rate 4.999999873689376e-05
802/802 ━━━━━━━━━━━━━━━━━━━━ 744s 921ms/step - accuracy: 0.9781 - auc: 0.9996 - loss: 0.0597 - val_accuracy: 0.9001 - val_auc: 0.9859 - val_loss: 0.3591 - learning_rate: 5.0000e-05
Epoch 3/10
802/802 ━━━━━━━━━━━━━━━━━━━━ 0s 738ms/step - accuracy: 0.9812 - auc: 0.9994 - loss: 0.0561마지막 학습률과 최소 손실 학습률이 저장되었

In [ ]:
# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")
print(f"Test AUC: {test_auc}")

251/251 ━━━━━━━━━━━━━━━━━━━━ 51s 203ms/step - accuracy: 0.8737 - auc: 0.9702 - loss: 0.5254
Test accuracy: 0.8861707448959351
Test loss: 0.487576425075531
Test AUC: 0.9743108749389648


In [ ]:
# 학습 종료 후 최적의 가중치로 모델 복원
if best_manager.latest_checkpoint:
    best_checkpoint.restore(best_manager.latest_checkpoint)
    print("학습 종료 후 최적의 가중치로 모델이 복원되었습니다.")
else:
    print("최적의 가중치를 찾을 수 없습니다.")


# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")
print(f"Test AUC: {test_auc}")

학습 종료 후 최적의 가중치로 모델이 복원되었습니다.
251/251 ━━━━━━━━━━━━━━━━━━━━ 36s 142ms/step - accuracy: 0.8634 - auc: 0.9675 - loss: 0.6239
Test accuracy: 0.8622066974639893
Test loss: 0.5986455082893372
Test AUC: 0.9686590433120728


In [ ]:
# 학습 종료 후 마지막 가중치로 모델 복원
if last_manager.latest_checkpoint:
    last_checkpoint.restore(last_manager.latest_checkpoint)
    print("학습 종료 후 마지막 가중치로 모델이 복원되었습니다.")
else:
    print("마지막 가중치를 찾을 수 없습니다.")

# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")
print(f"Test AUC: {test_auc}")

학습 종료 후 마지막 가중치로 모델이 복원되었습니다.
251/251 ━━━━━━━━━━━━━━━━━━━━ 36s 144ms/step - accuracy: 0.8737 - auc: 0.9702 - loss: 0.5254
Test accuracy: 0.8861707448959351
Test loss: 0.487576425075531
Test AUC: 0.9743108749389648


###모델학습 - 3차

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import ReduceLROnPlateau
import json

# 클래스 수 정의
NUM_CLASSES = 7

# EfficientNetB5 모델 정의
base_model = EfficientNetB5(weights='imagenet', include_top=False, input_shape=(456, 456, 3))

# 모델 설정
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

# 옵티마이저 설정
optimizer = AdamW(
    learning_rate=0.0001,
    weight_decay=1e-4,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-7
)

# ReduceLROnPlateau 스케줄러 설정
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6
)

# 모델 컴파일
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

# 학습률 저장 함수 (마지막 학습률과 최소 손실 학습률을 한 파일에 저장)
def save_learning_rates(last_lr, best_lr, filename='learning_rates.txt'):
    with open(filename, 'w') as f:
        f.write(f"last_learning_rate: {last_lr}\n")
        f.write(f"best_learning_rate: {best_lr}\n")
    print()
    print("마지막 학습률과 최소 손실 학습률이 저장되었습니다.")

# 학습률 복원 함수
def load_learning_rates(filename='learning_rates.txt'):
    try:
        with open(filename, 'r') as f:
            lines = f.readlines()
            last_lr = float(lines[0].split(": ")[1].strip())
            best_lr = float(lines[1].split(": ")[1].strip())
            print(f"학습률이 복원되었습니다: 마지막 학습률 = {last_lr}, 최소 손실 학습률 = {best_lr}")
            return last_lr, best_lr
    except FileNotFoundError:
        print("학습률 파일을 찾을 수 없습니다. 초기 학습률로 시작합니다.")
        return 0.0001, 0.0001  # 기본 학습률


# 체크포인트와 최적 검증 손실을 위한 변수 설정
best_checkpoint_path = 'best_model_checkpoint/'
last_checkpoint_path = 'last_model_checkpoint/'

best_checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)
last_checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)

best_manager = tf.train.CheckpointManager(best_checkpoint, best_checkpoint_path, max_to_keep=1)
last_manager = tf.train.CheckpointManager(last_checkpoint, last_checkpoint_path, max_to_keep=1)


# 마지막 학습 상태 복원
if last_manager.latest_checkpoint:
    last_checkpoint.restore(last_manager.latest_checkpoint)
    last_lr, best_lr = load_learning_rates()  # 학습률 복원
    optimizer.learning_rate.assign(last_lr)    # 옵티마이저에 마지막 학습률 적용

    print("마지막 학습 상태로 복원이 완료되었습니다.")
else:
    print("저장된 체크포인트가 없습니다. 새로 학습을 시작합니다.")

학습률이 복원되었습니다: 마지막 학습률 = 1.249999968422344e-05, 최소 손실 학습률 = 2.499999936844688e-05
마지막 학습 상태로 복원이 완료되었습니다.


In [ ]:
# 사용자 정의 콜백으로 최적 상태 저장
class CustomCheckpointCallback(tf.keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
        self.best_val_loss = float('inf')
        self.best_lr = 0.0001

    def on_epoch_end(self, epoch, logs=None):
        val_loss = logs.get('val_loss')
        current_lr = float(self.model.optimizer.learning_rate.numpy())

        # 최적 검증 손실 갱신 시 최적 상태와 학습률 저장
        if val_loss < self.best_val_loss:
            self.best_val_loss = val_loss
            self.best_lr = current_lr
            best_manager.save()  # 최적 상태 저장
            print()
            print(f"Best checkpoint saved at epoch {epoch+1} with val_loss {val_loss:.4f}")

        # 마지막 학습 상태 저장
        last_manager.save()

        # 마지막 학습률과 최소 손실 학습률 저장
        save_learning_rates(current_lr, self.best_lr or current_lr)
        print()
        print(f"Last checkpoint saved at epoch {epoch+1} with last learning rate {current_lr}")

# 모델 학습
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    verbose=1,
    callbacks=[reduce_lr, CustomCheckpointCallback()]  # 사용자 정의 체크포인트 콜백 추가
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


802/802 ━━━━━━━━━━━━━━━━━━━━ 0s 818ms/step - accuracy: 0.9950 - auc: 0.9998 - loss: 0.0180
Best checkpoint saved at epoch 1 with val_loss 0.4343

마지막 학습률과 최소 손실 학습률이 저장되었습니다.

Last checkpoint saved at epoch 1 with last learning rate 1.249999968422344e-05
802/802 ━━━━━━━━━━━━━━━━━━━━ 1026s 983ms/step - accuracy: 0.9950 - auc: 0.9998 - loss: 0.0180 - val_accuracy: 0.8983 - val_auc: 0.9799 - val_loss: 0.4343 - learning_rate: 1.2500e-05
Epoch 2/10
802/802 ━━━━━━━━━━━━━━━━━━━━ 0s 676ms/step - accuracy: 0.9944 - auc: 0.9996 - loss: 0.0223
Best checkpoint saved at epoch 2 with val_loss 0.4010

마지막 학습률과 최소 손실 학습률이 저장되었습니다.

Last checkpoint saved at epoch 2 with last learning rate 1.249999968422344e-05
802/802 ━━━━━━━━━━━━━━━━━━━━ 662s 819ms/step - accuracy: 0.9944 - auc: 0.9996 - loss: 0.0223 - val_accuracy: 0.9107 - val_auc: 0.9828 - val_loss: 0.4010 - learning_rate: 1.2500e-05
Epoch 3/10
802/802 ━━━━━━━━━━━━━━━━━━━━ 0s 684ms/step - accuracy: 0.9945 - auc: 1.0000 - loss: 0.0140
마지막 학습률과 최소 손실

KeyboardInterrupt: 

In [ ]:
# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")
print(f"Test AUC: {test_auc}")

251/251 ━━━━━━━━━━━━━━━━━━━━ 52s 208ms/step - accuracy: 0.8672 - auc: 0.9662 - loss: 0.6346
Test accuracy: 0.8741887211799622
Test loss: 0.6014552116394043
Test AUC: 0.9690998792648315


In [ ]:
# 학습 종료 후 최적의 가중치로 모델 복원
if best_manager.latest_checkpoint:
    best_checkpoint.restore(best_manager.latest_checkpoint)
    print("학습 종료 후 최적의 가중치로 모델이 복원되었습니다.")
else:
    print("최적의 가중치를 찾을 수 없습니다.")


# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")
print(f"Test AUC: {test_auc}")

학습 종료 후 최적의 가중치로 모델이 복원되었습니다.
251/251 ━━━━━━━━━━━━━━━━━━━━ 35s 138ms/step - accuracy: 0.8838 - auc: 0.9720 - loss: 0.5229
Test accuracy: 0.8896654844284058
Test loss: 0.5002182126045227
Test AUC: 0.9746838212013245


In [ ]:
# 학습 종료 후 마지막 가중치로 모델 복원
if last_manager.latest_checkpoint:
    last_checkpoint.restore(last_manager.latest_checkpoint)
    print("학습 종료 후 마지막 가중치로 모델이 복원되었습니다.")
else:
    print("마지막 가중치를 찾을 수 없습니다.")

# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")
print(f"Test AUC: {test_auc}")

학습 종료 후 마지막 가중치로 모델이 복원되었습니다.
251/251 ━━━━━━━━━━━━━━━━━━━━ 34s 136ms/step - accuracy: 0.8589 - auc: 0.9617 - loss: 0.7155
Test accuracy: 0.8622066974639893
Test loss: 0.6812511086463928
Test AUC: 0.9647322297096252


##10주차 - 연속학습을 위해 검증데이터 무결성 보장 후 재학습

###데이터 로드 + 필터링 안함


In [ ]:
# Kaggle API 설치
!pip install kaggle

# Kaggle API Key 설정
import json
import os
from google.colab import files
files.upload()  # kaggle.json 업로드

# Kaggle 설정
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 데이터셋 다운로드 (HAM10000 데이터셋)
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000

# 압축 해제
!unzip skin-cancer-mnist-ham10000.zip


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: ham10000_images_part_2/ISIC_0029325.jpg  
  inflating: ham10000_images_part_2/ISIC_0029326.jpg  
  inflating: ham10000_images_part_2/ISIC_0029327.jpg  
  inflating: ham10000_images_part_2/ISIC_0029328.jpg  
  inflating: ham10000_images_part_2/ISIC_0029329.jpg  
  inflating: ham10000_images_part_2/ISIC_0029330.jpg  
  inflating: ham10000_images_part_2/ISIC_0029331.jpg  
  inflating: ham10000_images_part_2/ISIC_0029332.jpg  
  inflating: ham10000_images_part_2/ISIC_0029333.jpg  
  inflating: ham10000_images_part_2/ISIC_0029334.jpg  
  inflating: ham10000_images_part_2/ISIC_0029335.jpg  
  inflating: ham10000_images_part_2/ISIC_0029336.jpg  
  inflating: ham10000_images_part_2/ISIC_0029337.jpg  
  inflating: ham10000_images_part_2/ISIC_0029338.jpg  
  inflating: ham10000_images_part_2/ISIC_0029339.jpg  
  inflating: ham10000_images_part_2/ISIC_0029340.jpg  
  inflating: ham10000_images_part_2/ISIC_0029341.jpg  
  inflating: ham10000_images_

In [ ]:
import os
import pandas as pd

# CSV 파일을 불러와서 DataFrame으로 저장
metadata_df = pd.read_csv('./HAM10000_metadata.csv')

# 이미지 폴더 경로
image_folder_1 = './HAM10000_images_part_1/'
image_folder_2 = './HAM10000_images_part_2/'

# 이미지 파일 경로 생성
metadata_df['image_path'] = metadata_df['image_id'].apply(
    lambda x: os.path.join(image_folder_1, f"{x}.jpg")
              if os.path.exists(os.path.join(image_folder_1, f"{x}.jpg"))
              else os.path.join(image_folder_2, f"{x}.jpg"))

# 메타데이터에 이미지 경로 추가 확인
print(metadata_df[['image_id', 'image_path']].head())


       image_id                                 image_path
0  ISIC_0027419  ./HAM10000_images_part_1/ISIC_0027419.jpg
1  ISIC_0025030  ./HAM10000_images_part_1/ISIC_0025030.jpg
2  ISIC_0026769  ./HAM10000_images_part_1/ISIC_0026769.jpg
3  ISIC_0025661  ./HAM10000_images_part_1/ISIC_0025661.jpg
4  ISIC_0031633  ./HAM10000_images_part_2/ISIC_0031633.jpg


In [ ]:
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# 고유한 클래스 추출 및 클래스 개수 계산
unique_classes = metadata_df['dx'].unique()
NUM_CLASSES = len(unique_classes)

print(f"Number of classes: {NUM_CLASSES}")
print(f"Classes: {unique_classes}")

# 메타데이터에서 데이터와 레이블 추출 (train/test로 변경)
train_df, test_df = train_test_split(metadata_df, test_size=0.2, stratify=metadata_df['dx'], random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['dx'], random_state=42)

# 학습 데이터용 ImageDataGenerator (데이터 증강 + 학습/검증 분리)
train_datagen = ImageDataGenerator(
    rescale=1./255,                # 정규화
    rotation_range=60,             # 이미지를 랜덤하게 40도까지 회전
    width_shift_range=0.2,         # 이미지의 가로 위치를 최대 20%까지 이동
    height_shift_range=0.2,        # 이미지의 세로 위치를 최대 20%까지 이동
    shear_range=0.2,               # 랜덤한 전단 변환 (shear)
    zoom_range=0.2,                # 랜덤하게 이미지 크기를 20%까지 확대/축소
    horizontal_flip=True,          # 이미지를 좌우로 랜덤하게 반전
    fill_mode='nearest'           # 이미지 이동 시 생기는 빈 공간을 채우는 방식
)

# 검증 데이터용 ImageDataGenerator (데이터 증강 없음, 정규화만 적용)
val_test_datagen = ImageDataGenerator(
    rescale=1./255
)

# 학습 데이터 생성기 (학습용)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,
    x_col='image_path',
    y_col='dx',
    target_size=(456, 456),
    batch_size=8,
    class_mode='categorical',
    shuffle=True
)

# 학습 데이터 생성기 (검증용)
val_generator = val_test_datagen.flow_from_dataframe(
    dataframe= val_df,
    directory=None,
    x_col='image_path',
    y_col='dx',
    target_size=(456, 456),
    batch_size=8,
    class_mode='categorical',
    shuffle=False
)

# 테스트 데이터 생성기
test_generator = val_test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=None,
    x_col='image_path',
    y_col='dx',
    target_size=(456, 456),
    batch_size=8,
    class_mode='categorical',
    shuffle=False  # 테스트 데이터는 shuffle하지 않음
)

# 생성기 출력 테스트
print(f"Training data shape: {train_generator.image_shape}")
print(f"Validation data shape: {val_generator.image_shape}")
print(f"Test data shape: {test_generator.image_shape}")

Number of classes: 7
Classes: ['bkl' 'nv' 'df' 'mel' 'vasc' 'bcc' 'akiec']
Found 6409 validated image filenames belonging to 7 classes.
Found 1603 validated image filenames belonging to 7 classes.
Found 2003 validated image filenames belonging to 7 classes.
Training data shape: (456, 456, 3)
Validation data shape: (456, 456, 3)
Test data shape: (456, 456, 3)


###모델학습 - 1차

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import ReduceLROnPlateau
import json

# 클래스 수 정의
NUM_CLASSES = 7

# EfficientNetB5 모델 정의
base_model = EfficientNetB5(weights='imagenet', include_top=False, input_shape=(456, 456, 3))

# 모델 설정
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

# 옵티마이저 설정
optimizer = AdamW(
    learning_rate=0.0001,
    weight_decay=1e-4,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-7
)

# ReduceLROnPlateau 스케줄러 설정
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6
)

# 모델 컴파일
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

# 학습률 저장 함수 (마지막 학습률과 최소 손실 학습률을 한 파일에 저장)
def save_learning_rates(last_lr, best_lr, filename='learning_rates.txt'):
    with open(filename, 'w') as f:
        f.write(f"last_learning_rate: {last_lr}\n")
        f.write(f"best_learning_rate: {best_lr}\n")
    print()
    print("마지막 학습률과 최소 손실 학습률이 저장되었습니다.")

# 학습률 복원 함수
def load_learning_rates(filename='learning_rates.txt'):
    try:
        with open(filename, 'r') as f:
            lines = f.readlines()
            last_lr = float(lines[0].split(": ")[1].strip())
            best_lr = float(lines[1].split(": ")[1].strip())
            print(f"학습률이 복원되었습니다: 마지막 학습률 = {last_lr}, 최소 손실 학습률 = {best_lr}")
            return last_lr, best_lr
    except FileNotFoundError:
        print("학습률 파일을 찾을 수 없습니다. 초기 학습률로 시작합니다.")
        return 0.0001, 0.0001  # 기본 학습률


# 체크포인트와 최적 검증 손실을 위한 변수 설정
best_checkpoint_path = 'best_model_checkpoint/'
last_checkpoint_path = 'last_model_checkpoint/'

best_checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)
last_checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)

best_manager = tf.train.CheckpointManager(best_checkpoint, best_checkpoint_path, max_to_keep=1)
last_manager = tf.train.CheckpointManager(last_checkpoint, last_checkpoint_path, max_to_keep=1)


# # 마지막 학습 상태 복원
# if last_manager.latest_checkpoint:
#     last_checkpoint.restore(last_manager.latest_checkpoint)
#     last_lr, best_lr = load_learning_rates()  # 학습률 복원
#     optimizer.learning_rate.assign(last_lr)    # 옵티마이저에 마지막 학습률 적용

#     print("마지막 학습 상태로 복원이 완료되었습니다.")
# else:
#     print("저장된 체크포인트가 없습니다. 새로 학습을 시작합니다.")

115263384/115263384 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
# 사용자 정의 콜백으로 최적 상태 저장
class CustomCheckpointCallback(tf.keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
        self.best_val_loss = float('inf')
        self.best_lr = 0.0001

    def on_epoch_end(self, epoch, logs=None):
        val_loss = logs.get('val_loss')
        current_lr = float(self.model.optimizer.learning_rate.numpy())

        # 최적 검증 손실 갱신 시 최적 상태와 학습률 저장
        if val_loss < self.best_val_loss:
            self.best_val_loss = val_loss
            self.best_lr = current_lr
            best_manager.save()  # 최적 상태 저장
            print()
            print(f"Best checkpoint saved at epoch {epoch+1} with val_loss {val_loss:.4f}")

        # 마지막 학습 상태 저장
        last_manager.save()

        # 마지막 학습률과 최소 손실 학습률 저장
        save_learning_rates(current_lr, self.best_lr or current_lr)
        print()
        print(f"Last checkpoint saved at epoch {epoch+1} with last learning rate {current_lr}")

# 모델 학습
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    verbose=1,
    callbacks=[reduce_lr, CustomCheckpointCallback()]  # 사용자 정의 체크포인트 콜백 추가
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


802/802 ━━━━━━━━━━━━━━━━━━━━ 0s 890ms/step - accuracy: 0.6887 - auc: 0.9260 - loss: 0.9238
Best checkpoint saved at epoch 1 with val_loss 0.5898

마지막 학습률과 최소 손실 학습률이 저장되었습니다.

Last checkpoint saved at epoch 1 with last learning rate 9.999999747378752e-05
802/802 ━━━━━━━━━━━━━━━━━━━━ 1030s 974ms/step - accuracy: 0.6888 - auc: 0.9260 - loss: 0.9236 - val_accuracy: 0.7767 - val_auc: 0.9721 - val_loss: 0.5898 - learning_rate: 1.0000e-04
Epoch 2/10
802/802 ━━━━━━━━━━━━━━━━━━━━ 0s 712ms/step - accuracy: 0.8245 - auc: 0.9782 - loss: 0.5016
Best checkpoint saved at epoch 2 with val_loss 0.5516

마지막 학습률과 최소 손실 학습률이 저장되었습니다.

Last checkpoint saved at epoch 2 with last learning rate 9.999999747378752e-05
802/802 ━━━━━━━━━━━━━━━━━━━━ 631s 780ms/step - accuracy: 0.8245 - auc: 0.9782 - loss: 0.5015 - val_accuracy: 0.7898 - val_auc: 0.9735 - val_loss: 0.5516 - learning_rate: 1.0000e-04
Epoch 3/10
802/802 ━━━━━━━━━━━━━━━━━━━━ 0s 714ms/step - accuracy: 0.8644 - auc: 0.9863 - loss: 0.3859
마지막 학습률과 최소 손실

In [ ]:
# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")
print(f"Test AUC: {test_auc}")

251/251 ━━━━━━━━━━━━━━━━━━━━ 36s 142ms/step - accuracy: 0.8421 - auc: 0.9731 - loss: 0.5574
Test accuracy: 0.8387418985366821
Test loss: 0.5545221567153931
Test AUC: 0.9746241569519043


In [ ]:
# 학습 종료 후 최적의 가중치로 모델 복원
if best_manager.latest_checkpoint:
    best_checkpoint.restore(best_manager.latest_checkpoint)
    print("학습 종료 후 최적의 가중치로 모델이 복원되었습니다.")
else:
    print("최적의 가중치를 찾을 수 없습니다.")


# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")
print(f"Test AUC: {test_auc}")

학습 종료 후 최적의 가중치로 모델이 복원되었습니다.


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


251/251 ━━━━━━━━━━━━━━━━━━━━ 80s 205ms/step - accuracy: 0.8651 - auc: 0.9833 - loss: 0.4322
Test accuracy: 0.867199182510376
Test loss: 0.4147076904773712
Test AUC: 0.9839032888412476


In [ ]:
# 학습 종료 후 마지막 가중치로 모델 복원
if last_manager.latest_checkpoint:
    last_checkpoint.restore(last_manager.latest_checkpoint)
    print("학습 종료 후 마지막 가중치로 모델이 복원되었습니다.")
else:
    print("마지막 가중치를 찾을 수 없습니다.")

# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")
print(f"Test AUC: {test_auc}")

학습 종료 후 마지막 가중치로 모델이 복원되었습니다.
251/251 ━━━━━━━━━━━━━━━━━━━━ 38s 152ms/step - accuracy: 0.8421 - auc: 0.9731 - loss: 0.5574
Test accuracy: 0.8387418985366821
Test loss: 0.5545220971107483
Test AUC: 0.97462397813797


###모델학습 - 2차 최적 검증 포인트로 복원

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import ReduceLROnPlateau
import json

# 클래스 수 정의
NUM_CLASSES = 7

# EfficientNetB5 모델 정의
base_model = EfficientNetB5(weights='imagenet', include_top=False, input_shape=(456, 456, 3))

# 모델 설정
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

# 옵티마이저 설정
optimizer = AdamW(
    learning_rate=0.0001,
    weight_decay=1e-4,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-7
)

# ReduceLROnPlateau 스케줄러 설정
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6
)

# 모델 컴파일
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

# 학습률 저장 함수 (마지막 학습률과 최소 손실 학습률을 한 파일에 저장)
def save_learning_rates(last_lr, best_lr, filename='learning_rates.txt'):
    with open(filename, 'w') as f:
        f.write(f"last_learning_rate: {last_lr}\n")
        f.write(f"best_learning_rate: {best_lr}\n")
    print("학습률 저장 완료.")

# 학습률 복원 함수
def load_learning_rates(filename='learning_rates.txt'):
    try:
        with open(filename, 'r') as f:
            lines = f.readlines()
            last_lr = float(lines[0].split(": ")[1].strip())
            best_lr = float(lines[1].split(": ")[1].strip())
            print(f"학습률이 복원되었습니다: 마지막 학습률 = {last_lr}, 최소 손실 학습률 = {best_lr}")
            return last_lr, best_lr
    except FileNotFoundError:
        print("학습률 파일을 찾을 수 없습니다. 초기 학습률로 시작합니다.")
        return 0.0001, 0.0001  # 기본 학습률


# 체크포인트와 최적 검증 손실을 위한 변수 설정
best_checkpoint_path = 'best_model_checkpoint/'
last_checkpoint_path = 'last_model_checkpoint/'

best_checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)
last_checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)

best_manager = tf.train.CheckpointManager(best_checkpoint, best_checkpoint_path, max_to_keep=1)
last_manager = tf.train.CheckpointManager(last_checkpoint, last_checkpoint_path, max_to_keep=1)

# 최적 검증 손실 체크포인트 복원
if best_manager.latest_checkpoint:
    best_checkpoint.restore(best_manager.latest_checkpoint)
    best_lr = load_learning_rates()  # 최적 검증 손실 시 학습률 복원
    optimizer.learning_rate.assign(0.0001)  # 옵티마이저에 복원된 학습률 적용
    print("최적 검증 손실 체크포인트로 복원이 완료되었습니다.")
else:
    print("저장된 최적 검증 손실 체크포인트가 없습니다.")

115263384/115263384 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
학습률 파일을 찾을 수 없습니다. 초기 학습률로 시작합니다.
최적 검증 손실 체크포인트로 복원이 완료되었습니다.


In [ ]:
# 사용자 정의 콜백으로 최적 상태 저장
class CustomCheckpointCallback(tf.keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
        self.best_val_loss = 0.4472
        self.best_lr = 0.0001

    def on_epoch_end(self, epoch, logs=None):
        val_loss = logs.get('val_loss')
        current_lr = float(self.model.optimizer.learning_rate.numpy())

        # 최적 검증 손실 갱신 시 최적 상태와 학습률 저장
        if val_loss < self.best_val_loss:
            self.best_val_loss = val_loss
            self.best_lr = current_lr
            best_manager.save()  # 최적 상태 저장
            print()
            print(f"Best checkpoint saved at epoch {epoch+1} with val_loss {val_loss:.4f}")

        # 마지막 학습 상태 저장
        last_manager.save()

        # 마지막 학습률과 최소 손실 학습률 저장
        save_learning_rates(current_lr, self.best_lr or current_lr)
        print(f"Last checkpoint saved at epoch {epoch+1} with last learning rate {current_lr}")

# 모델 학습
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    verbose=1,
    callbacks=[reduce_lr, CustomCheckpointCallback()]  # 사용자 정의 체크포인트 콜백 추가
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


802/802 ━━━━━━━━━━━━━━━━━━━━ 0s 867ms/step - accuracy: 0.9000 - auc: 0.9922 - loss: 0.2763학습률 저장 완료.
Last checkpoint saved at epoch 1 with last learning rate 9.999999747378752e-05
802/802 ━━━━━━━━━━━━━━━━━━━━ 990s 944ms/step - accuracy: 0.9000 - auc: 0.9922 - loss: 0.2763 - val_accuracy: 0.6656 - val_auc: 0.9406 - val_loss: 0.9169 - learning_rate: 1.0000e-04
Epoch 2/10
802/802 ━━━━━━━━━━━━━━━━━━━━ 0s 672ms/step - accuracy: 0.9229 - auc: 0.9953 - loss: 0.2176학습률 저장 완료.
Last checkpoint saved at epoch 2 with last learning rate 9.999999747378752e-05
802/802 ━━━━━━━━━━━━━━━━━━━━ 583s 721ms/step - accuracy: 0.9229 - auc: 0.9953 - loss: 0.2177 - val_accuracy: 0.7966 - val_auc: 0.9696 - val_loss: 0.6231 - learning_rate: 1.0000e-04
Epoch 3/10
802/802 ━━━━━━━━━━━━━━━━━━━━ 0s 670ms/step - accuracy: 0.9263 - auc: 0.9952 - loss: 0.2101학습률 저장 완료.
Last checkpoint saved at epoch 3 with last learning rate 9.999999747378752e-05
802/802 ━━━━━━━━━━━━━━━━━━━━ 582s 720ms/step - accuracy: 0.9263 - auc: 0.995

KeyboardInterrupt: 

In [ ]:
# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")
print(f"Test AUC: {test_auc}")

251/251 ━━━━━━━━━━━━━━━━━━━━ 51s 201ms/step - accuracy: 0.8093 - auc: 0.9697 - loss: 0.6472
Test accuracy: 0.8212680816650391
Test loss: 0.6098374128341675
Test AUC: 0.9720614552497864


In [ ]:
# 학습 종료 후 최적의 가중치로 모델 복원
if best_manager.latest_checkpoint:
    best_checkpoint.restore(best_manager.latest_checkpoint)
    print("학습 종료 후 최적의 가중치로 모델이 복원되었습니다.")
else:
    print("최적의 가중치를 찾을 수 없습니다.")


# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")
print(f"Test AUC: {test_auc}")

학습 종료 후 최적의 가중치로 모델이 복원되었습니다.
251/251 ━━━━━━━━━━━━━━━━━━━━ 44s 176ms/step - accuracy: 0.8651 - auc: 0.9833 - loss: 0.4322
Test accuracy: 0.867199182510376
Test loss: 0.4147077202796936
Test AUC: 0.9839032888412476


In [ ]:
# 학습 종료 후 마지막 가중치로 모델 복원
if last_manager.latest_checkpoint:
    last_checkpoint.restore(last_manager.latest_checkpoint)
    print("학습 종료 후 마지막 가중치로 모델이 복원되었습니다.")
else:
    print("마지막 가중치를 찾을 수 없습니다.")

# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")
print(f"Test AUC: {test_auc}")

학습 종료 후 마지막 가중치로 모델이 복원되었습니다.
251/251 ━━━━━━━━━━━━━━━━━━━━ 39s 155ms/step - accuracy: 0.8093 - auc: 0.9697 - loss: 0.6472
Test accuracy: 0.8212680816650391
Test loss: 0.6098374128341675
Test AUC: 0.9720614552497864


##클래스 통합본 테스트 (채택안함)

###데이터 로드 + 필터링 안함


In [ ]:
# Kaggle API 설치
!pip install kaggle

# Kaggle API Key 설정
import json
import os
from google.colab import files
files.upload()  # kaggle.json 업로드

# Kaggle 설정
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 데이터셋 다운로드 (HAM10000 데이터셋)
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000

# 압축 해제
!unzip skin-cancer-mnist-ham10000.zip


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: ham10000_images_part_2/ISIC_0029326.jpg  
  inflating: ham10000_images_part_2/ISIC_0029327.jpg  
  inflating: ham10000_images_part_2/ISIC_0029328.jpg  
  inflating: ham10000_images_part_2/ISIC_0029329.jpg  
  inflating: ham10000_images_part_2/ISIC_0029330.jpg  
  inflating: ham10000_images_part_2/ISIC_0029331.jpg  
  inflating: ham10000_images_part_2/ISIC_0029332.jpg  
  inflating: ham10000_images_part_2/ISIC_0029333.jpg  
  inflating: ham10000_images_part_2/ISIC_0029334.jpg  
  inflating: ham10000_images_part_2/ISIC_0029335.jpg  
  inflating: ham10000_images_part_2/ISIC_0029336.jpg  
  inflating: ham10000_images_part_2/ISIC_0029337.jpg  
  inflating: ham10000_images_part_2/ISIC_0029338.jpg  
  inflating: ham10000_images_part_2/ISIC_0029339.jpg  
  inflating: ham10000_images_part_2/ISIC_0029340.jpg  
  inflating: ham10000_images_part_2/ISIC_0029341.jpg  
  inflating: ham10000_images_part_2/ISIC_0029342.jpg  
  inflating: ham10000_images_

In [ ]:
import os
import pandas as pd

# CSV 파일을 불러와서 DataFrame으로 저장
metadata_df = pd.read_csv('./HAM10000_metadata.csv')

# 이미지 폴더 경로
image_folder_1 = './HAM10000_images_part_1/'
image_folder_2 = './HAM10000_images_part_2/'

# 이미지 파일 경로 생성
metadata_df['image_path'] = metadata_df['image_id'].apply(
    lambda x: os.path.join(image_folder_1, f"{x}.jpg")
              if os.path.exists(os.path.join(image_folder_1, f"{x}.jpg"))
              else os.path.join(image_folder_2, f"{x}.jpg"))

# 메타데이터에 이미지 경로 추가 확인
print(metadata_df[['image_id', 'image_path']].head())


       image_id                                 image_path
0  ISIC_0027419  ./HAM10000_images_part_1/ISIC_0027419.jpg
1  ISIC_0025030  ./HAM10000_images_part_1/ISIC_0025030.jpg
2  ISIC_0026769  ./HAM10000_images_part_1/ISIC_0026769.jpg
3  ISIC_0025661  ./HAM10000_images_part_1/ISIC_0025661.jpg
4  ISIC_0031633  ./HAM10000_images_part_2/ISIC_0031633.jpg


In [ ]:
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# 고유한 클래스 추출 및 클래스 개수 계산
unique_classes = metadata_df['dx'].unique()
NUM_CLASSES = len(unique_classes)

print(f"Number of classes: {NUM_CLASSES}")
print(f"Classes: {unique_classes}")

# 메타데이터에서 데이터와 레이블 추출 (train/test로 변경)
train_df, test_df = train_test_split(metadata_df, test_size=0.2, stratify=metadata_df['dx'], random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['dx'], random_state=42)

# 학습 데이터용 ImageDataGenerator (데이터 증강 + 학습/검증 분리)
train_datagen = ImageDataGenerator(
    rescale=1./255,                # 정규화
    rotation_range=60,             # 이미지를 랜덤하게 40도까지 회전
    width_shift_range=0.2,         # 이미지의 가로 위치를 최대 20%까지 이동
    height_shift_range=0.2,        # 이미지의 세로 위치를 최대 20%까지 이동
    shear_range=0.2,               # 랜덤한 전단 변환 (shear)
    zoom_range=0.2,                # 랜덤하게 이미지 크기를 20%까지 확대/축소
    horizontal_flip=True,          # 이미지를 좌우로 랜덤하게 반전
    fill_mode='nearest'           # 이미지 이동 시 생기는 빈 공간을 채우는 방식
)

# 검증 데이터용 ImageDataGenerator (데이터 증강 없음, 정규화만 적용)
val_test_datagen = ImageDataGenerator(
    rescale=1./255
)

# 학습 데이터 생성기 (학습용)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,
    x_col='image_path',
    y_col='dx',
    target_size=(456, 456),
    batch_size=8,
    class_mode='categorical',
    shuffle=True
)

# 학습 데이터 생성기 (검증용)
val_generator = val_test_datagen.flow_from_dataframe(
    dataframe= val_df,
    directory=None,
    x_col='image_path',
    y_col='dx',
    target_size=(456, 456),
    batch_size=8,
    class_mode='categorical',
    shuffle=False
)

# 테스트 데이터 생성기
test_generator = val_test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=None,
    x_col='image_path',
    y_col='dx',
    target_size=(456, 456),
    batch_size=8,
    class_mode='categorical',
    shuffle=False  # 테스트 데이터는 shuffle하지 않음
)

# 생성기 출력 테스트
print(f"Training data shape: {train_generator.image_shape}")
print(f"Validation data shape: {val_generator.image_shape}")
print(f"Test data shape: {test_generator.image_shape}")

Number of classes: 7
Classes: ['bkl' 'nv' 'df' 'mel' 'vasc' 'bcc' 'akiec']
Found 6409 validated image filenames belonging to 7 classes.
Found 1603 validated image filenames belonging to 7 classes.
Found 2003 validated image filenames belonging to 7 classes.
Training data shape: (456, 456, 3)
Validation data shape: (456, 456, 3)
Test data shape: (456, 456, 3)


###모델학습 - 1차

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import ReduceLROnPlateau
import json
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2

# 클래스 수 정의
NUM_CLASSES = 7

# EfficientNetB5 모델 정의
base_model = EfficientNetB5(weights='imagenet', include_top=False, input_shape=(456, 456, 3))

# 모델 설정
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dense(NUM_CLASSES, activation='softmax')
])

# 옵티마이저 설정
optimizer = AdamW(
    learning_rate=0.0001,
    weight_decay=1e-4,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-7
)

# ReduceLROnPlateau 스케줄러 설정
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-6
)

# 모델 컴파일
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

# 학습률 저장 함수 (마지막 학습률과 최소 손실 학습률을 한 파일에 저장)
def save_learning_rates(last_lr, best_lr, filename='learning_rates.txt'):
    with open(filename, 'w') as f:
        f.write(f"last_learning_rate: {last_lr}\n")
        f.write(f"best_learning_rate: {best_lr}\n")
    print("학습률 저장! 마지막 학습률과 최소 손실 학습률이 저장되었습니다.")

# 학습률 복원 함수
def load_learning_rates(filename='learning_rates.txt'):
    try:
        with open(filename, 'r') as f:
            lines = f.readlines()
            last_lr = float(lines[0].split(": ")[1].strip())
            best_lr = float(lines[1].split(": ")[1].strip())
            print(f"학습률이 복원되었습니다: 마지막 학습률 = {last_lr}, 최소 손실 학습률 = {best_lr}")
            return last_lr, best_lr
    except FileNotFoundError:
        print("학습률 파일을 찾을 수 없습니다. 초기 학습률로 시작합니다.")
        return 0.0001, 0.0001  # 기본 학습률


# 체크포인트와 최적 검증 손실을 위한 변수 설정
best_checkpoint_path = 'best_model_checkpoint/'
last_checkpoint_path = 'last_model_checkpoint/'

best_checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)
last_checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)

best_manager = tf.train.CheckpointManager(best_checkpoint, best_checkpoint_path, max_to_keep=1)
last_manager = tf.train.CheckpointManager(last_checkpoint, last_checkpoint_path, max_to_keep=1)


# # 마지막 학습 상태 복원
# if last_manager.latest_checkpoint:
#     last_checkpoint.restore(last_manager.latest_checkpoint)
#     last_lr, best_lr = load_learning_rates()  # 학습률 복원
#     optimizer.learning_rate.assign(last_lr)    # 옵티마이저에 마지막 학습률 적용

#     print("마지막 학습 상태로 복원이 완료되었습니다.")
# else:
#     print("저장된 체크포인트가 없습니다. 새로 학습을 시작합니다.")

In [ ]:
# 사용자 정의 콜백으로 최적 상태 저장
class CustomCheckpointCallback(tf.keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
        self.best_val_loss = float('inf')
        self.best_lr = 0.0001

    def on_epoch_end(self, epoch, logs=None):
        val_loss = logs.get('val_loss')
        current_lr = float(self.model.optimizer.learning_rate.numpy())

        # 최적 검증 손실 갱신 시 최적 상태와 학습률 저장
        if val_loss < self.best_val_loss:
            self.best_val_loss = val_loss
            self.best_lr = current_lr
            best_manager.save()  # 최적 상태 저장
            print()
            print(f"Best checkpoint saved at epoch {epoch+1} with val_loss {val_loss:.4f}")

        # 마지막 학습 상태 저장
        last_manager.save()

        # 마지막 학습률과 최소 손실 학습률 저장
        save_learning_rates(current_lr, self.best_lr or current_lr)
        print(f"Last checkpoint saved at epoch {epoch+1} with last learning rate {current_lr}")

# 모델 학습
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    verbose=1,
    callbacks=[reduce_lr, CustomCheckpointCallback()]  # 사용자 정의 체크포인트 콜백 추가
)

Epoch 1/20
802/802 ━━━━━━━━━━━━━━━━━━━━ 0s 832ms/step - accuracy: 0.5370 - auc: 0.7985 - loss: 3.9492
Best checkpoint saved at epoch 1 with val_loss 2.9815
학습률 저장! 마지막 학습률과 최소 손실 학습률이 저장되었습니다.
Last checkpoint saved at epoch 1 with last learning rate 9.999999747378752e-05
802/802 ━━━━━━━━━━━━━━━━━━━━ 955s 922ms/step - accuracy: 0.5372 - auc: 0.7986 - loss: 3.9487 - val_accuracy: 0.7355 - val_auc: 0.9290 - val_loss: 2.9815 - learning_rate: 1.0000e-04
Epoch 2/20
802/802 ━━━━━━━━━━━━━━━━━━━━ 0s 653ms/step - accuracy: 0.7365 - auc: 0.9314 - loss: 2.9224
Best checkpoint saved at epoch 2 with val_loss 2.5767
학습률 저장! 마지막 학습률과 최소 손실 학습률이 저장되었습니다.
Last checkpoint saved at epoch 2 with last learning rate 9.999999747378752e-05
802/802 ━━━━━━━━━━━━━━━━━━━━ 580s 718ms/step - accuracy: 0.7365 - auc: 0.9314 - loss: 2.9222 - val_accuracy: 0.7567 - val_auc: 0.9320 - val_loss: 2.5767 - learning_rate: 1.0000e-04
Epoch 3/20
802/802 ━━━━━━━━━━━━━━━━━━━━ 0s 652ms/step - accuracy: 0.7877 - auc: 0.9518 - loss:

In [ ]:
# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")
print(f"Test AUC: {test_auc}")

In [ ]:
# 학습 종료 후 최적의 가중치로 모델 복원
if best_manager.latest_checkpoint:
    best_checkpoint.restore(best_manager.latest_checkpoint)
    print("학습 종료 후 최적의 가중치로 모델이 복원되었습니다.")
else:
    print("최적의 가중치를 찾을 수 없습니다.")


# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")
print(f"Test AUC: {test_auc}")

학습 종료 후 최적의 가중치로 모델이 복원되었습니다.
251/251 ━━━━━━━━━━━━━━━━━━━━ 55s 217ms/step - accuracy: 0.7808 - auc: 0.9630 - loss: 0.9775
Test accuracy: 0.7848227620124817
Test loss: 0.9647805690765381
Test AUC: 0.9632596969604492


In [ ]:
# 학습 종료 후 마지막 가중치로 모델 복원
if last_manager.latest_checkpoint:
    last_checkpoint.restore(last_manager.latest_checkpoint)
    print("학습 종료 후 마지막 가중치로 모델이 복원되었습니다.")
else:
    print("마지막 가중치를 찾을 수 없습니다.")

# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")
print(f"Test AUC: {test_auc}")

학습 종료 후 마지막 가중치로 모델이 복원되었습니다.
251/251 ━━━━━━━━━━━━━━━━━━━━ 39s 156ms/step - accuracy: 0.7681 - auc: 0.9556 - loss: 1.0183
Test accuracy: 0.7703444957733154
Test loss: 1.0148954391479492
Test AUC: 0.9548154473304749


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import ReduceLROnPlateau
import json
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2

# 클래스 수 정의
NUM_CLASSES = 7

# EfficientNetB5 모델 정의
base_model = EfficientNetB5(weights='imagenet', include_top=False, input_shape=(456, 456, 3))

# 모델 설정
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dense(NUM_CLASSES, activation='softmax')
])

# 옵티마이저 설정
optimizer = AdamW(
    learning_rate=0.0001,
    weight_decay=1e-4,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-7
)

# ReduceLROnPlateau 스케줄러 설정
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6
)

# 모델 컴파일
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

# 학습률 저장 함수 (마지막 학습률과 최소 손실 학습률을 한 파일에 저장)
def save_learning_rates(last_lr, best_lr, filename='learning_rates.txt'):
    with open(filename, 'w') as f:
        f.write(f"last_learning_rate: {last_lr}\n")
        f.write(f"best_learning_rate: {best_lr}\n")
    print("학습률 저장! 마지막 학습률과 최소 손실 학습률이 저장되었습니다.")

# 학습률 복원 함수
def load_learning_rates(filename='learning_rates.txt'):
    try:
        with open(filename, 'r') as f:
            lines = f.readlines()
            last_lr = float(lines[0].split(": ")[1].strip())
            best_lr = float(lines[1].split(": ")[1].strip())
            print(f"학습률이 복원되었습니다: 마지막 학습률 = {last_lr}, 최소 손실 학습률 = {best_lr}")
            return last_lr, best_lr
    except FileNotFoundError:
        print("학습률 파일을 찾을 수 없습니다. 초기 학습률로 시작합니다.")
        return 0.0001, 0.0001  # 기본 학습률


# 체크포인트와 최적 검증 손실을 위한 변수 설정
best_checkpoint_path = 'best_model_checkpoint/'
last_checkpoint_path = 'last_model_checkpoint/'

best_checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)
last_checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)

best_manager = tf.train.CheckpointManager(best_checkpoint, best_checkpoint_path, max_to_keep=1)
last_manager = tf.train.CheckpointManager(last_checkpoint, last_checkpoint_path, max_to_keep=1)


# # 마지막 학습 상태 복원
# if last_manager.latest_checkpoint:
#     last_checkpoint.restore(last_manager.latest_checkpoint)
#     last_lr, best_lr = load_learning_rates()  # 학습률 복원
#     optimizer.learning_rate.assign(last_lr)    # 옵티마이저에 마지막 학습률 적용

#     print("마지막 학습 상태로 복원이 완료되었습니다.")
# else:
#     print("저장된 체크포인트가 없습니다. 새로 학습을 시작합니다.")

# 최적 검증 손실 상태 복원
if best_manager.latest_checkpoint:
    best_checkpoint.restore(best_manager.latest_checkpoint)
    last_lr, best_lr = load_learning_rates()  # 최적 검증 손실 시 학습률 복원
    optimizer.learning_rate.assign(best_lr)    # 옵티마이저에 최적 학습률 적용

    print("최적 검증 손실 체크포인트로 복원이 완료되었습니다.")
else:
    print("저장된 최적 체크포인트가 없습니다. 새로 학습을 시작합니다.")

학습률이 복원되었습니다: 마지막 학습률 = 9.999999747378752e-05, 최소 손실 학습률 = 9.999999747378752e-05
최적 검증 손실 체크포인트로 복원이 완료되었습니다.


In [ ]:
# 사용자 정의 콜백으로 최적 상태 저장
class CustomCheckpointCallback(tf.keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
        self.best_val_loss = 1.0329 ##최소 로스값 수동입력
        self.best_lr = 0.0001

    def on_epoch_end(self, epoch, logs=None):
        val_loss = logs.get('val_loss')
        current_lr = float(self.model.optimizer.learning_rate.numpy())

        # 최적 검증 손실 갱신 시 최적 상태와 학습률 저장
        if val_loss < self.best_val_loss:
            self.best_val_loss = val_loss
            self.best_lr = current_lr
            best_manager.save()  # 최적 상태 저장
            print()
            print(f"Best checkpoint saved at epoch {epoch+1} with val_loss {val_loss:.4f}")

        # 마지막 학습 상태 저장
        last_manager.save()

        # 마지막 학습률과 최소 손실 학습률 저장
        save_learning_rates(current_lr, self.best_lr or current_lr)
        print(f"Last checkpoint saved at epoch {epoch+1} with last learning rate {current_lr}")

# 모델 학습
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    verbose=1,
    callbacks=[reduce_lr, CustomCheckpointCallback()]  # 사용자 정의 체크포인트 콜백 추가
)

Epoch 1/10
802/802 ━━━━━━━━━━━━━━━━━━━━ 0s 897ms/step - accuracy: 0.9021 - auc: 0.9914 - loss: 0.5615


Best checkpoint saved at epoch 1 with val_loss 0.7249
학습률 저장! 마지막 학습률과 최소 손실 학습률이 저장되었습니다.
Last checkpoint saved at epoch 1 with last learning rate 9.999999747378752e-05
802/802 ━━━━━━━━━━━━━━━━━━━━ 982s 1s/step - accuracy: 0.9021 - auc: 0.9914 - loss: 0.5615 - val_accuracy: 0.8353 - val_auc: 0.9779 - val_loss: 0.7249 - learning_rate: 1.0000e-04
Epoch 2/10
802/802 ━━━━━━━━━━━━━━━━━━━━ 0s 712ms/step - accuracy: 0.9018 - auc: 0.9920 - loss: 0.4909
Best checkpoint saved at epoch 2 with val_loss 0.6854
학습률 저장! 마지막 학습률과 최소 손실 학습률이 저장되었습니다.
Last checkpoint saved at epoch 2 with last learning rate 9.999999747378752e-05
802/802 ━━━━━━━━━━━━━━━━━━━━ 623s 773ms/step - accuracy: 0.9018 - auc: 0.9920 - loss: 0.4908 - val_accuracy: 0.8372 - val_auc: 0.9765 - val_loss: 0.6854 - learning_rate: 1.0000e-04
Epoch 3/10
802/802 ━━━━━━━━━━━━━━━━━━━━ 0s 705ms/step - accuracy: 0.9239 - auc: 0.9943 - loss: 0.3897
Best checkpoint saved at epoch 3 with val_loss 0.6314
학습률 저장! 마지막 학습률과 최소 손실 학습률이 저장되었습니다.
Last 

In [ ]:
# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")
print(f"Test AUC: {test_auc}")

251/251 ━━━━━━━━━━━━━━━━━━━━ 56s 218ms/step - accuracy: 0.7997 - auc: 0.9494 - loss: 0.9237
Test accuracy: 0.7978032827377319
Test loss: 0.8853890299797058
Test AUC: 0.9531782269477844


In [ ]:
# 학습 종료 후 최적의 가중치로 모델 복원
if best_manager.latest_checkpoint:
    best_checkpoint.restore(best_manager.latest_checkpoint)
    print("학습 종료 후 최적의 가중치로 모델이 복원되었습니다.")
else:
    print("최적의 가중치를 찾을 수 없습니다.")


# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")
print(f"Test AUC: {test_auc}")

학습 종료 후 최적의 가중치로 모델이 복원되었습니다.
251/251 ━━━━━━━━━━━━━━━━━━━━ 37s 147ms/step - accuracy: 0.8775 - auc: 0.9750 - loss: 0.5074
Test accuracy: 0.8741887211799622
Test loss: 0.5191161036491394
Test AUC: 0.9756788611412048


In [ ]:
# 학습 종료 후 마지막 가중치로 모델 복원
if last_manager.latest_checkpoint:
    last_checkpoint.restore(last_manager.latest_checkpoint)
    print("학습 종료 후 마지막 가중치로 모델이 복원되었습니다.")
else:
    print("마지막 가중치를 찾을 수 없습니다.")

# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")
print(f"Test AUC: {test_auc}")

학습 종료 후 마지막 가중치로 모델이 복원되었습니다.
251/251 ━━━━━━━━━━━━━━━━━━━━ 38s 150ms/step - accuracy: 0.7997 - auc: 0.9494 - loss: 0.9237
Test accuracy: 0.7978032827377319
Test loss: 0.8853890299797058
Test AUC: 0.9531782269477844


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import ReduceLROnPlateau
import json
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2

# 클래스 수 정의
NUM_CLASSES = 7

# EfficientNetB5 모델 정의
base_model = EfficientNetB5(weights='imagenet', include_top=False, input_shape=(456, 456, 3))

# 모델 설정
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dense(NUM_CLASSES, activation='softmax')
])

# 옵티마이저 설정
optimizer = AdamW(
    learning_rate=0.0001,
    weight_decay=1e-4,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-7
)

# ReduceLROnPlateau 스케줄러 설정
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6
)

# 모델 컴파일
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

# 학습률 저장 함수 (마지막 학습률과 최소 손실 학습률을 한 파일에 저장)
def save_learning_rates(last_lr, best_lr, filename='learning_rates.txt'):
    with open(filename, 'w') as f:
        f.write(f"last_learning_rate: {last_lr}\n")
        f.write(f"best_learning_rate: {best_lr}\n")
    print("학습률 저장! 마지막 학습률과 최소 손실 학습률이 저장되었습니다.")

# 학습률 복원 함수
def load_learning_rates(filename='learning_rates.txt'):
    try:
        with open(filename, 'r') as f:
            lines = f.readlines()
            last_lr = float(lines[0].split(": ")[1].strip())
            best_lr = float(lines[1].split(": ")[1].strip())
            print(f"학습률이 복원되었습니다: 마지막 학습률 = {last_lr}, 최소 손실 학습률 = {best_lr}")
            return last_lr, best_lr
    except FileNotFoundError:
        print("학습률 파일을 찾을 수 없습니다. 초기 학습률로 시작합니다.")
        return 0.0001, 0.0001  # 기본 학습률


# 체크포인트와 최적 검증 손실을 위한 변수 설정
best_checkpoint_path = 'best_model_checkpoint/'
last_checkpoint_path = 'last_model_checkpoint/'

best_checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)
last_checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)

best_manager = tf.train.CheckpointManager(best_checkpoint, best_checkpoint_path, max_to_keep=1)
last_manager = tf.train.CheckpointManager(last_checkpoint, last_checkpoint_path, max_to_keep=1)


# # 마지막 학습 상태 복원
# if last_manager.latest_checkpoint:
#     last_checkpoint.restore(last_manager.latest_checkpoint)
#     last_lr, best_lr = load_learning_rates()  # 학습률 복원
#     optimizer.learning_rate.assign(last_lr)    # 옵티마이저에 마지막 학습률 적용

#     print("마지막 학습 상태로 복원이 완료되었습니다.")
# else:
#     print("저장된 체크포인트가 없습니다. 새로 학습을 시작합니다.")

# 최적 검증 손실 상태 복원
if best_manager.latest_checkpoint:
    best_checkpoint.restore(best_manager.latest_checkpoint)
    last_lr, best_lr = load_learning_rates()  # 최적 검증 손실 시 학습률 복원
    optimizer.learning_rate.assign(best_lr)    # 옵티마이저에 최적 학습률 적용

    print("최적 검증 손실 체크포인트로 복원이 완료되었습니다.")
else:
    print("저장된 최적 체크포인트가 없습니다. 새로 학습을 시작합니다.")

IndexError: Read less bytes than requested

In [ ]:
# 사용자 정의 콜백으로 최적 상태 저장
class CustomCheckpointCallback(tf.keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
        self.best_val_loss = 0.5452 ##최소 로스값 수동입력
        self.best_lr = 0.0001

    def on_epoch_end(self, epoch, logs=None):
        val_loss = logs.get('val_loss')
        current_lr = float(self.model.optimizer.learning_rate.numpy())

        # 최적 검증 손실 갱신 시 최적 상태와 학습률 저장
        if val_loss < self.best_val_loss:
            self.best_val_loss = val_loss
            self.best_lr = current_lr
            best_manager.save()  # 최적 상태 저장
            print()
            print(f"Best checkpoint saved at epoch {epoch+1} with val_loss {val_loss:.4f}")

        # 마지막 학습 상태 저장
        last_manager.save()

        # 마지막 학습률과 최소 손실 학습률 저장
        save_learning_rates(current_lr, self.best_lr or current_lr)
        print(f"Last checkpoint saved at epoch {epoch+1} with last learning rate {current_lr}")

# 모델 학습
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    verbose=1,
    callbacks=[reduce_lr, CustomCheckpointCallback()]  # 사용자 정의 체크포인트 콜백 추가
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


802/802 ━━━━━━━━━━━━━━━━━━━━ 0s 878ms/step - accuracy: 0.9534 - auc: 0.9976 - loss: 0.1709학습률 저장! 마지막 학습률과 최소 손실 학습률이 저장되었습니다.
Last checkpoint saved at epoch 1 with last learning rate 9.999999747378752e-05
802/802 ━━━━━━━━━━━━━━━━━━━━ 1006s 956ms/step - accuracy: 0.9534 - auc: 0.9976 - loss: 0.1709 - val_accuracy: 0.7954 - val_auc: 0.9581 - val_loss: 0.9024 - learning_rate: 1.0000e-04
Epoch 2/10
802/802 ━━━━━━━━━━━━━━━━━━━━ 0s 694ms/step - accuracy: 0.9521 - auc: 0.9975 - loss: 0.1736학습률 저장! 마지막 학습률과 최소 손실 학습률이 저장되었습니다.
Last checkpoint saved at epoch 2 with last learning rate 9.999999747378752e-05
802/802 ━━━━━━━━━━━━━━━━━━━━ 622s 742ms/step - accuracy: 0.9522 - auc: 0.9975 - loss: 0.1736 - val_accuracy: 0.8528 - val_auc: 0.9743 - val_loss: 0.5734 - learning_rate: 1.0000e-04
Epoch 3/10
802/802 ━━━━━━━━━━━━━━━━━━━━ 0s 684ms/step - accuracy: 0.9658 - auc: 0.9977 - loss: 0.1495학습률 저장! 마지막 학습률과 최소 손실 학습률이 저장되었습니다.
Last checkpoint saved at epoch 3 with last learning rate 9.999999747378752e-

KeyboardInterrupt: 

In [ ]:
# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")
print(f"Test AUC: {test_auc}")

251/251 ━━━━━━━━━━━━━━━━━━━━ 43s 170ms/step - accuracy: 0.8080 - auc: 0.9628 - loss: 0.7245
Test accuracy: 0.8122815489768982
Test loss: 0.6990258693695068
Test AUC: 0.9643623232841492


In [ ]:
# 학습 종료 후 최적의 가중치로 모델 복원
if best_manager.latest_checkpoint:
    best_checkpoint.restore(best_manager.latest_checkpoint)
    print("학습 종료 후 최적의 가중치로 모델이 복원되었습니다.")
else:
    print("최적의 가중치를 찾을 수 없습니다.")


# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")
print(f"Test AUC: {test_auc}")

학습 종료 후 최적의 가중치로 모델이 복원되었습니다.
251/251 ━━━━━━━━━━━━━━━━━━━━ 38s 150ms/step - accuracy: 0.7997 - auc: 0.9494 - loss: 0.9237
Test accuracy: 0.7978032827377319
Test loss: 0.8853890299797058
Test AUC: 0.9531782269477844


In [ ]:
# 학습 종료 후 마지막 가중치로 모델 복원
if last_manager.latest_checkpoint:
    last_checkpoint.restore(last_manager.latest_checkpoint)
    print("학습 종료 후 마지막 가중치로 모델이 복원되었습니다.")
else:
    print("마지막 가중치를 찾을 수 없습니다.")

# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")
print(f"Test AUC: {test_auc}")

학습 종료 후 마지막 가중치로 모델이 복원되었습니다.
251/251 ━━━━━━━━━━━━━━━━━━━━ 38s 150ms/step - accuracy: 0.7997 - auc: 0.9494 - loss: 0.9237
Test accuracy: 0.7978032827377319
Test loss: 0.8853890299797058
Test AUC: 0.9531782269477844


##11주차 모델 픽스 후

###데이터 로드 + 필터링 안함


In [ ]:
# Kaggle API 설치
!pip install kaggle

# Kaggle API Key 설정
import json
import os
from google.colab import files
files.upload()  # kaggle.json 업로드

# Kaggle 설정
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 데이터셋 다운로드 (HAM10000 데이터셋)
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000

# 압축 해제
!unzip skin-cancer-mnist-ham10000.zip


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: ham10000_images_part_2/ISIC_0029325.jpg  
  inflating: ham10000_images_part_2/ISIC_0029326.jpg  
  inflating: ham10000_images_part_2/ISIC_0029327.jpg  
  inflating: ham10000_images_part_2/ISIC_0029328.jpg  
  inflating: ham10000_images_part_2/ISIC_0029329.jpg  
  inflating: ham10000_images_part_2/ISIC_0029330.jpg  
  inflating: ham10000_images_part_2/ISIC_0029331.jpg  
  inflating: ham10000_images_part_2/ISIC_0029332.jpg  
  inflating: ham10000_images_part_2/ISIC_0029333.jpg  
  inflating: ham10000_images_part_2/ISIC_0029334.jpg  
  inflating: ham10000_images_part_2/ISIC_0029335.jpg  
  inflating: ham10000_images_part_2/ISIC_0029336.jpg  
  inflating: ham10000_images_part_2/ISIC_0029337.jpg  
  inflating: ham10000_images_part_2/ISIC_0029338.jpg  
  inflating: ham10000_images_part_2/ISIC_0029339.jpg  
  inflating: ham10000_images_part_2/ISIC_0029340.jpg  
  inflating: ham10000_images_part_2/ISIC_0029341.jpg  
  inflating: ham10000_images_

In [ ]:
import os
import pandas as pd

# CSV 파일을 불러와서 DataFrame으로 저장
metadata_df = pd.read_csv('./HAM10000_metadata.csv')

# 이미지 폴더 경로
image_folder_1 = './HAM10000_images_part_1/'
image_folder_2 = './HAM10000_images_part_2/'

# 이미지 파일 경로 생성
metadata_df['image_path'] = metadata_df['image_id'].apply(
    lambda x: os.path.join(image_folder_1, f"{x}.jpg")
              if os.path.exists(os.path.join(image_folder_1, f"{x}.jpg"))
              else os.path.join(image_folder_2, f"{x}.jpg"))

# 메타데이터에 이미지 경로 추가 확인
print(metadata_df[['image_id', 'image_path']].head())


       image_id                                 image_path
0  ISIC_0027419  ./HAM10000_images_part_1/ISIC_0027419.jpg
1  ISIC_0025030  ./HAM10000_images_part_1/ISIC_0025030.jpg
2  ISIC_0026769  ./HAM10000_images_part_1/ISIC_0026769.jpg
3  ISIC_0025661  ./HAM10000_images_part_1/ISIC_0025661.jpg
4  ISIC_0031633  ./HAM10000_images_part_2/ISIC_0031633.jpg


In [ ]:
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# 고유한 클래스 추출 및 클래스 개수 계산
unique_classes = metadata_df['dx'].unique()
NUM_CLASSES = len(unique_classes)

print(f"Number of classes: {NUM_CLASSES}")
print(f"Classes: {unique_classes}")

# 메타데이터에서 데이터와 레이블 추출 (train/test로 변경)
train_df, test_df = train_test_split(metadata_df, test_size=0.2, stratify=metadata_df['dx'], random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['dx'], random_state=42)

# 학습 데이터용 ImageDataGenerator (데이터 증강 + 학습/검증 분리)
train_datagen = ImageDataGenerator(
    rescale=1./255,                # 정규화
    rotation_range=60,             # 이미지를 랜덤하게 40도까지 회전
    width_shift_range=0.2,         # 이미지의 가로 위치를 최대 20%까지 이동
    height_shift_range=0.2,        # 이미지의 세로 위치를 최대 20%까지 이동
    shear_range=0.2,               # 랜덤한 전단 변환 (shear)
    zoom_range=0.2,                # 랜덤하게 이미지 크기를 20%까지 확대/축소
    horizontal_flip=True,          # 이미지를 좌우로 랜덤하게 반전
    fill_mode='nearest'           # 이미지 이동 시 생기는 빈 공간을 채우는 방식
)

# 검증 데이터용 ImageDataGenerator (데이터 증강 없음, 정규화만 적용)
val_test_datagen = ImageDataGenerator(
    rescale=1./255
)

# 학습 데이터 생성기 (학습용)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=None,
    x_col='image_path',
    y_col='dx',
    target_size=(456, 456),
    batch_size=8,
    class_mode='categorical',
    shuffle=True
)

# # 학습 데이터 생성기 (검증용)
val_generator = val_test_datagen.flow_from_dataframe(
    dataframe= val_df,
    directory=None,
    x_col='image_path',
    y_col='dx',
    target_size=(456, 456),
    batch_size=8,
    class_mode='categorical',
    shuffle=False
)

# 테스트 데이터 생성기
test_generator = val_test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=None,
    x_col='image_path',
    y_col='dx',
    target_size=(456, 456),
    batch_size=8,
    class_mode='categorical',
    shuffle=False  # 테스트 데이터는 shuffle하지 않음
)

# 생성기 출력 테스트
print(f"Training data shape: {train_generator.image_shape}")
print(f"Validation data shape: {val_generator.image_shape}")
print(f"Test data shape: {test_generator.image_shape}")

Number of classes: 7
Classes: ['bkl' 'nv' 'df' 'mel' 'vasc' 'bcc' 'akiec']
Found 6409 validated image filenames belonging to 7 classes.
Found 1603 validated image filenames belonging to 7 classes.
Found 2003 validated image filenames belonging to 7 classes.
Training data shape: (456, 456, 3)
Validation data shape: (456, 456, 3)
Test data shape: (456, 456, 3)


###모델학습 - 1차

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import ReduceLROnPlateau
import json

# 클래스 수 정의
NUM_CLASSES = 7

# EfficientNetB5 모델 정의
base_model = EfficientNetB5(weights='imagenet', include_top=False, input_shape=(456, 456, 3))

# 모델 설정
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])

# 옵티마이저 설정
optimizer = AdamW(
    learning_rate=0.0001,
    weight_decay=1e-4,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-7
)

# ReduceLROnPlateau 스케줄러 설정
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6
)

# 모델 컴파일
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

# 학습률 저장 함수 (마지막 학습률과 최소 손실 학습률을 한 파일에 저장)
def save_learning_rates(last_lr, best_lr, filename='learning_rates.txt'):
    with open(filename, 'w') as f:
        f.write(f"last_learning_rate: {last_lr}\n")
        f.write(f"best_learning_rate: {best_lr}\n")
    print()
    print("마지막 학습률과 최소 손실 학습률이 저장되었습니다.")

# 학습률 복원 함수
def load_learning_rates(filename='learning_rates.txt'):
    try:
        with open(filename, 'r') as f:
            lines = f.readlines()
            last_lr = float(lines[0].split(": ")[1].strip())
            best_lr = float(lines[1].split(": ")[1].strip())
            print(f"학습률이 복원되었습니다: 마지막 학습률 = {last_lr}, 최소 손실 학습률 = {best_lr}")
            return last_lr, best_lr
    except FileNotFoundError:
        print("학습률 파일을 찾을 수 없습니다. 초기 학습률로 시작합니다.")
        return 0.0001, 0.0001  # 기본 학습률


# 체크포인트와 최적 검증 손실을 위한 변수 설정
best_checkpoint_path = 'best_model_checkpoint/'
last_checkpoint_path = 'last_model_checkpoint/'

best_checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)
last_checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)

best_manager = tf.train.CheckpointManager(best_checkpoint, best_checkpoint_path, max_to_keep=1)
last_manager = tf.train.CheckpointManager(last_checkpoint, last_checkpoint_path, max_to_keep=1)


# # 마지막 학습 상태 복원
# if last_manager.latest_checkpoint:
#     last_checkpoint.restore(last_manager.latest_checkpoint)
#     last_lr, best_lr = load_learning_rates()  # 학습률 복원
#     optimizer.learning_rate.assign(last_lr)    # 옵티마이저에 마지막 학습률 적용

#     print("마지막 학습 상태로 복원이 완료되었습니다.")
# else:
#     print("저장된 체크포인트가 없습니다. 새로 학습을 시작합니다.")

115263384/115263384 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
# 사용자 정의 콜백으로 최적 상태 저장
class CustomCheckpointCallback(tf.keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
        self.best_val_loss = float('inf')
        self.best_lr = 0.0001

    def on_epoch_end(self, epoch, logs=None):
        val_loss = logs.get('val_loss')
        current_lr = float(self.model.optimizer.learning_rate.numpy())

        # 최적 검증 손실 갱신 시 최적 상태와 학습률 저장
        if val_loss < self.best_val_loss:
            self.best_val_loss = val_loss
            self.best_lr = current_lr
            best_manager.save()  # 최적 상태 저장
            print()
            print(f"Best checkpoint saved at epoch {epoch+1} with val_loss {val_loss:.4f}")

        # 마지막 학습 상태 저장
        last_manager.save()

        # 마지막 학습률과 최소 손실 학습률 저장
        save_learning_rates(current_lr, self.best_lr or current_lr)
        print()
        print(f"Last checkpoint saved at epoch {epoch+1} with last learning rate {current_lr}")

# 모델 학습
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    verbose=1,
    callbacks=[reduce_lr, CustomCheckpointCallback()]  # 사용자 정의 체크포인트 콜백 추가
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


802/802 ━━━━━━━━━━━━━━━━━━━━ 0s 890ms/step - accuracy: 0.6887 - auc: 0.9260 - loss: 0.9238
Best checkpoint saved at epoch 1 with val_loss 0.5898

마지막 학습률과 최소 손실 학습률이 저장되었습니다.

Last checkpoint saved at epoch 1 with last learning rate 9.999999747378752e-05
802/802 ━━━━━━━━━━━━━━━━━━━━ 1030s 974ms/step - accuracy: 0.6888 - auc: 0.9260 - loss: 0.9236 - val_accuracy: 0.7767 - val_auc: 0.9721 - val_loss: 0.5898 - learning_rate: 1.0000e-04
Epoch 2/10
802/802 ━━━━━━━━━━━━━━━━━━━━ 0s 712ms/step - accuracy: 0.8245 - auc: 0.9782 - loss: 0.5016
Best checkpoint saved at epoch 2 with val_loss 0.5516

마지막 학습률과 최소 손실 학습률이 저장되었습니다.

Last checkpoint saved at epoch 2 with last learning rate 9.999999747378752e-05
802/802 ━━━━━━━━━━━━━━━━━━━━ 631s 780ms/step - accuracy: 0.8245 - auc: 0.9782 - loss: 0.5015 - val_accuracy: 0.7898 - val_auc: 0.9735 - val_loss: 0.5516 - learning_rate: 1.0000e-04
Epoch 3/10
802/802 ━━━━━━━━━━━━━━━━━━━━ 0s 714ms/step - accuracy: 0.8644 - auc: 0.9863 - loss: 0.3859
마지막 학습률과 최소 손실

In [ ]:
# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")
print(f"Test AUC: {test_auc}")

251/251 ━━━━━━━━━━━━━━━━━━━━ 36s 142ms/step - accuracy: 0.8421 - auc: 0.9731 - loss: 0.5574
Test accuracy: 0.8387418985366821
Test loss: 0.5545221567153931
Test AUC: 0.9746241569519043


In [ ]:
# 학습 종료 후 최적의 가중치로 모델 복원
if best_manager.latest_checkpoint:
    best_checkpoint.restore(best_manager.latest_checkpoint)
    print("학습 종료 후 최적의 가중치로 모델이 복원되었습니다.")
else:
    print("최적의 가중치를 찾을 수 없습니다.")


# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")
print(f"Test AUC: {test_auc}")

학습 종료 후 최적의 가중치로 모델이 복원되었습니다.


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


251/251 ━━━━━━━━━━━━━━━━━━━━ 80s 205ms/step - accuracy: 0.8651 - auc: 0.9833 - loss: 0.4322
Test accuracy: 0.867199182510376
Test loss: 0.4147076904773712
Test AUC: 0.9839032888412476


In [ ]:
# 학습 종료 후 마지막 가중치로 모델 복원
if last_manager.latest_checkpoint:
    last_checkpoint.restore(last_manager.latest_checkpoint)
    print("학습 종료 후 마지막 가중치로 모델이 복원되었습니다.")
else:
    print("마지막 가중치를 찾을 수 없습니다.")

# 모델 평가
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")
print(f"Test loss: {test_loss}")
print(f"Test AUC: {test_auc}")

학습 종료 후 마지막 가중치로 모델이 복원되었습니다.
251/251 ━━━━━━━━━━━━━━━━━━━━ 38s 152ms/step - accuracy: 0.8421 - auc: 0.9731 - loss: 0.5574
Test accuracy: 0.8387418985366821
Test loss: 0.5545220971107483
Test AUC: 0.97462397813797


In [ ]:
import tensorflow as tf
print(f"현재 TensorFlow 버전: {tf.__version__}")


현재 TensorFlow 버전: 2.17.1
